In [1]:
# from bokeh.io import output_notebook
# from bokeh.resources import INLINE

In [2]:
# output_notebook(resources=INLINE)

In [14]:
from cobra.io import read_legacy_sbml, read_sbml_model
from cobra import Metabolite

In [10]:
from cobra import Reaction

In [11]:
import imp
foo = imp.load_source('curation_tools','/Users/clie/Desktop/EFPro2/Functions_Modules/curation_tools.py')

In [16]:
model = read_sbml_model('iCL_Conversion3.xml')

In [17]:
print model.objective

Maximize
-1.0*BIOMASS_REACTION_reverse_42726 + 1.0*BIOMASS_REACTION


In [20]:
model.reactions.EX_glc__D_e.bounds = -20, 1000
print model.optimize().f

0.99396112705


In [21]:
print model.optimize().f

0.99396112705


In [22]:
for rxn in model.reactions:
    if rxn.id.startswith('EX_') and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_o2_e: o2_e <=>  -1000.0 1000.0 -8.05442235359
EX_co2_e: co2_e <=>  -1000.0 1000.0 -10.1284067319
EX_nh4_e: nh4_e <=>  -1000.0 1000.0 -7.11701015996
EX_pi_e: pi_e <=>  -1000.0 1000.0 -2.39747399689
EX_so4_e: so4_e <=>  -1000.0 1000.0 -0.0815048124181
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 30.2798031634
EX_nac_e: nac_e <=>  -1000.0 1000.0 0.157314227578
EX_glc__D_e: glc__D_e <=>  -20 1000 -20.0
EX_mal__L_e: mal__L_e -->  0.0 1000.0 20.5878845209
EX_fum_e: fum_e -->  0.0 1000.0 0.174937158361


In [26]:
solution = model.optimize()

In [27]:
import escher

In [29]:
maps = escher.Builder(
                    model=model,
                    map_name='Ustilago maydis 521 genome-scale metabolic model',
                    map_json='UstiMap10.json',
                    show_gene_reaction_rules=True,
                    use_3d_transform = True)

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_notebook()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Implement corrected Phospholipid Synthesis Pathways!

#### Phosphatidate Pathway : Using 16:0 at C1 and 16:0 C2

Berg J, Tymoczko JL, Stryer L (2006). Biochemistry (6th ed.). San Francisco: W. H. Freeman. ISBN 0-7167-8724-5.

In [30]:
rxn = model.reactions.G3PD1

In [31]:
rxn = model.reactions.GLYC3PDEHYDROGBIOSYN__45__RXN__45__GLYCEROL__45__3P__47__NADP__47____47__DIHYDROXY__45__ACETONE__45__PHOSPHATE__47__NADPH__47__PROTON__46__59__46__
rxn.id = 'G3PD2'

In [32]:
met = model.metabolites.get_by_id('Palmitoyl__45__ACPs_c')
met.formula = 'C27H51N2O8PRS'
met.charge = -1
met.id = 'palmACP_c'

In [33]:
rxn = model.reactions.RXN__45__10462
rxn.build_reaction_from_string('glyc3p_c + palmACP_c <=> 1hdecg3p_c + ACP_c')
rxn.name = 'Glycerol-3-phosphate acyltransferase (C16:0)'
rxn.id = 'G3PAT160'

unknown metabolite '1hdecg3p_c' created


In [34]:
met = model.metabolites.get_by_id('1hdecg3p_c')
met.name='1-hexadecanoyl-sn-glycerol 3-phosphate'
met.formula='C19H37O7P'
met.charge=-2

In [35]:
rxn = model.reactions._1__45__ACYLGLYCEROL__45__3__45__P__45__ACYLTRANSFER__45__RXN
rxn.build_reaction_from_string('1hdecg3p_c + palmACP_c -> ACP_c + pa160_c')
rxn.name = '1-hexadecanoyl-sn-glycerol 3-phosphate O-acyltransferase (n-C16:0)'
rxn.id = 'AGPAT160'

In [36]:
met =  model.metabolites.CDPDIACYLGLYCEROL_c
met.name='CDP-1,2-dihexadecanoylglycerol'
met.formula='C44H79N3O15P2'
met.charge=-2

In [37]:
met.id = 'cdpdhdecg_c'

In [38]:
rxn = model.reactions.CDPDIGLYSYN__45__RXN
rxn.build_reaction_from_string('ctp_c + h_c + pa160_c -> cdpdhdecg_c + ppi_c')
rxn.name = 'CDP-diacylglycerol synthetase (n-C16:0)'
rxn.id = 'DASYN160'

In [39]:
met = model.metabolites.pgp_EC_c
met.name='Phosphatidylglycerophosphate (dihexadecanoyl, n-C16:0)'
met.formula='C38H73O13P2'
met.charge=-3
met.id = 'pgp160_c'

In [40]:
rxn = model.reactions.PHOSPHAGLYPSYN__45__RXN
rxn.name = 'Phosphatidylglycerol synthase (n-C16:0)'
rxn.id = 'PGSA160'

In [41]:
met = model.metabolites.L__45__1__45__PHOSPHATIDYL__45__GLYCEROL_c
met.name='Phosphatidylglycerol (dihexadecanoyl, n-C16:0)'
met.formula='C38H74O10P1'
met.charge=-1
met.id = 'pg160_c'

In [42]:
rxn = model.reactions.PGPPHOSPHA__45__RXN
rxn.name = 'Phosphatidylglycerol phosphate phosphatase (n-C16:0)'
rxn.id = 'PGPP160'

In [43]:
met = model.metabolites.L__45__1__45__PHOSPHATIDYL__45__SERINE_c
met.name='Phosphatidylserine (dihexadecanoyl, n-C16:0)'
met.formula='C38H73N1O10P1'
met.charge=0
met.id = 'ps160_c'

In [44]:
rxn = model.reactions.PHOSPHASERSYN__45__RXN
rxn.name = 'Phosphatidylserine syntase (n-C16:0)'
rxn.id = 'PSSA160'

In [45]:
met = model.metabolites.L__45__1__45__PHOSPHATIDYL__45__ETHANOLAMINE_c
met.name='Phosphatidylethanolamine (dihexadecanoyl, n-C16:0)'
met.formula='C37H74N1O8P1'
met.charge=0
met.id = 'pe160_c'

In [46]:
rxn = model.reactions.PHOSPHASERDECARB__45__RXN
rxn.name = 'Phosphatidylserine decarboxylase (n-C16:0)'
rxn.id = 'PSD160'

In [47]:
met = model.metabolites.CPD__45__405_c
met.name='Phosphatidyl N methylethanolamine (dihexadecanoyl, n-C16:0)'
met.formula='C38H76N1O8P1'
met.charge=0
met.id = 'ptdmeeta160_c'

In [48]:
rxn = model.reactions._2__46__1__46__1__46__17__45__RXN
rxn.name = 'Phosphatidylethanolamine N methyltransferase (n-C16:0)'
rxn.id = 'PETOHM'

In [49]:
met = model.metabolites.ptd2meeta_SC_c
met.name='Phosphatidyl N dimethylethanolamine (dihexadecanoyl, n-C16:0)'
met.formula='C39H78N1O8P1'
met.charge=0
met.id = 'ptd2meeta160_c'

In [50]:
rxn = model.reactions._2__46__1__46__1__46__71__45__RXN
rxn.name = 'Methylene fatty acyl phospholipid synthase (n-C16:0)'
rxn.id = 'MFAPS'

In [51]:
met = model.metabolites.PHOSPHATIDYLCHOLINE_c
met.name='Phosphatidylcholine (dihexadecanoyl, n-C16:0)'
met.formula='C40H80N1O8P1'
met.charge=0
met.id = 'pc160_c'

In [52]:
rxn = model.reactions.RXN4FS__45__2
rxn.name = 'Phosphatidyl N methylethanolamine N methyltransferase (n-C16:0)'
rxn.id = 'PMETM'

### Balancing of Chitin Synthesis

In [53]:
met = model.metabolites.chtn_c
met.name='Chitin monomer C8H13NO5'
met.formula='C8H13NO5'
met.charge=0
met.id = 'chitin_c'

In [54]:
rxn = model.reactions.CHITIN__45__SYNTHASE__45__RXN
rxn.name = 'Chitin synthase'
rxn.id = 'CHTNS'

In [55]:
model.reactions.CHTNS.build_reaction_from_string('uacgam_c -> h_c + udp_c + chitin_c')

Quick check for growth

In [57]:
model.optimize().f

0.9978769544121785

In [59]:
from cobra.flux_analysis import find_essential_reactions

In [60]:
essential_reactions = find_essential_reactions(model)
unbalanced_list = foo.find_unbalanced_reactions(model)
check_and_fix = [x.id for x in essential_reactions if x.id in unbalanced_list[0]]
for x in check_and_fix:
    print x,unbalanced_list[0][x]

There are 338 unbalanced reactions and 0 metabolites with a faulty syntax
3OAS180 {'C': -1.0, 'H': -9.0, 'charge': -1.0, 'O': 11.0, 'N': 2.0, 'P': 1.0, 'S': 1.0, 'R': 1.0}
EAR120y {'C': 0, 'H': -2.0, 'charge': 0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
MCOATA {'C': -14.0, 'H': -22.0, 'charge': 1.0, 'O': -10.0, 'N': -2.0, 'P': -1.0, 'S': -1.0, 'R': -1.0}
RXN__45__9548 {'C': 29.0, 'H': 55.0, 'charge': -1.0, 'O': 8.0, 'N': 2.0, 'P': 1.0, 'S': 1.0, 'R': 1.0}
EAR160y {'C': 27.0, 'H': 49.0, 'charge': -1.0, 'O': 8.0, 'N': 2.0, 'P': 1.0, 'S': 1.0, 'R': 1.0}
3OAS60 {'C': 12.0, 'H': 20.0, 'charge': -1.0, 'O': 9.0, 'N': 2.0, 'P': 1.0, 'S': 1.0, 'R': 1.0}
EAR140y {'C': 25.0, 'H': 45.0, 'charge': -1.0, 'O': 8.0, 'N': 2.0, 'P': 1.0, 'S': 1.0, 'R': 1.0}
BIOMASS_REACTION {'C': -44.37737999999989, 'H': -128.8003400000002, 'charge': -44.89506000000003, 'O': -19.711309999999877, 'N': -5.5009199999999945, 'P': -0.9892799999999615, 'S': -0.082, 'R': 0}
3OAR160 {'C': 0, 'H': -2.0, 'charge': 0, 'O': 0, 'N': 

# Fix unbalanced essential reactions!!

In [61]:
import re

In [62]:
foo.find_unbalanced_reactions(model)

There are 338 unbalanced reactions and 0 metabolites with a faulty syntax


({'3HAD100': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD120': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD140': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD160': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD180': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD40': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD60': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3HAD80': {'C': 0,
   'H': 2.0,
   'N': 0,
   'O': 1.0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3OAR100': {'C': 0,
   'H': -2.0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3OAR120': {'C': 0,
 

In [66]:
model2 = read_sbml_model("/Users/clie/Desktop/Side_Projects/Memote/Ecoli_Model_collection/iJO1366/iJO1366.xml")

In [67]:
for met in model.metabolites:
    if met.formula == '' or met.formula is None:
        if met.id in model2.metabolites:
            met.formula = model2.metabolites.get_by_id(met.id).formula
            if met.charge is None and model2.metabolites.get_by_id(met.id).charge is not None:
                met.charge = model2.metabolites.get_by_id(met.id).charge

In [68]:
foo.find_unbalanced_reactions(model)

There are 308 unbalanced reactions and 0 metabolites with a faulty syntax


({'3HAD160': {'C': 27.0,
   'H': 51.0,
   'N': 2.0,
   'O': 9.0,
   'P': 1.0,
   'R': 1.0,
   'S': 1.0,
   'charge': 0},
  '3HAD60': {'C': 17.0,
   'H': 31.0,
   'N': 2.0,
   'O': 9.0,
   'P': 1.0,
   'R': 1.0,
   'S': 1.0,
   'charge': 0},
  '3OAR160': {'C': 0,
   'H': -2.0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  '3OAR60': {'C': -17.0,
   'H': -31.0,
   'N': -2.0,
   'O': -9.0,
   'P': -1.0,
   'R': -1.0,
   'S': -1.0,
   'charge': 0},
  '3OAS120': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': -1.0},
  '3OAS140': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': -1.0},
  '3OAS160': {'C': -27.0,
   'H': -49.0,
   'N': -2.0,
   'O': -9.0,
   'P': -1.0,
   'R': -1.0,
   'S': -1.0,
   'charge': 0},
  '3OAS180': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': -1.0},
  '3OAS60': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R':

In [69]:
foo.superprint(model,'3HAD160')

R__45__3__45__Hydroxypalmitoyl__45__ACPs_c --> h2o_c + tpalm2eACP_c 

R__45__3__45__Hydroxypalmitoyl__45__ACPs_c None 0
<=>
tpalm2eACP_c C27H49N2O8PRS 0
h2o_c H2O 0


In [70]:
met = model.metabolites.get_by_id('R__45__3__45__Hydroxypalmitoyl__45__ACPs_c')
met.formula = 'C27H51N2O9PRS'
met.charge = -1
met.id = '3hpalmACP_c'
met = model.metabolites.get_by_id('tpalm2eACP_c')
met.charge = -1

In [71]:
foo.superprint(model,'3HAD60')

R__45__3__45__hydroxyhexanoyl__45__ACPs_c --> h2o_c + thex2eACP_c 

R__45__3__45__hydroxyhexanoyl__45__ACPs_c None 0
<=>
h2o_c H2O 0
thex2eACP_c C17H29N2O8PRS 0


In [72]:
met = model.metabolites.get_by_id('R__45__3__45__hydroxyhexanoyl__45__ACPs_c')
met.formula = 'C17H31N2O9PRS'
met.charge = -1
met.id = '3hhexACP'
met = model.metabolites.get_by_id('thex2eACP_c')
met.charge = -1

In [73]:
foo.superprint(model,'3OAR160')

_3__45__oxo__45__ealmitoyl__45__ACPs_c + h_c + nadph_c --> 3hpalmACP_c + nadp_c 

_3__45__oxo__45__ealmitoyl__45__ACPs_c None 0
nadph_c C21H26N7O17P3 -4
h_c H 1
<=>
nadp_c C21H25N7O17P3 -3
3hpalmACP_c C27H51N2O9PRS -1


In [74]:
met = model.metabolites.get_by_id('_3__45__oxo__45__ealmitoyl__45__ACPs_c')
met.formula = 'C27H49N2O9PRS'
met.charge = -1
met.id = '3opalmACP'

In [75]:
foo.superprint(model,'3OAR60')

3ohexACP_c + h_c + nadph_c --> 3hhexACP + nadp_c 

nadph_c C21H26N7O17P3 -4
h_c H 1
3ohexACP_c C17H29N2O9PRS 0
<=>
nadp_c C21H25N7O17P3 -3
3hhexACP C17H31N2O9PRS -1


In [76]:
met = model.metabolites.get_by_id('3ohexACP_c')
met.charge = -1

In [77]:
foo.superprint(model,'3OAS100')

h_c + malACP_c + ocACP_c --> 3odecACP_c + ACP_c + co2_c 

malACP_c C14H22N2O10PRS -1
h_c H 1
ocACP_c C19H35N2O8PRS -1
<=>
3odecACP_c C21H37N2O9PRS 0
ACP_c C11H21N2O7PRS -1
co2_c CO2 0


In [78]:
met = model.metabolites.get_by_id('malACP_c')
met.charge = -2
met = model.metabolites.get_by_id('3odecACP_c')
met.charge = -1

In [79]:
foo.find_unbalanced_reactions(model)

There are 299 unbalanced reactions and 0 metabolites with a faulty syntax


({'3OAR100': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 1.0},
  '3OAS180': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 1.0},
  '3OAS60': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': -1.0},
  '3OAS80': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 1.0},
  'ACOATA': {'C': 0,
   'H': 0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 1.0},
  'ACP_REPLENISHMENT_RXN': {'C': -13.0,
   'H': -23.0,
   'N': -2.0,
   'O': -8.0,
   'P': -1.0,
   'R': -1.0,
   'S': -1.0,
   'charge': 1.0},
  'ACYLCOADEHYDROG__45__RXN': {'C': 0,
   'H': -1.0,
   'N': 0,
   'O': 0,
   'P': 0,
   'R': 0,
   'S': 0,
   'charge': 0},
  'ACYLCOASYN__45__RXN__45__POLYMER__45__INST__45__CPD66__45__39__45__C10__45__H20__47__CO__45__A__47__ATP__47____47__TETRADECANOYL__45__COA__47__PPI__47__AMP__46__66__46__': {'C': 0,
   'H': 0,
   

In [80]:
foo.superprint(model,'3OAS180')

coa_c + h_c + 2.0 malACP_c + palmACP_c --> 3ooctdACP_c + 2.0 ACP_c + co2_c + malcoa_c 

malACP_c C14H22N2O10PRS -2
coa_c C21H32N7O16P3S -4
h_c H 1
palmACP_c C27H51N2O8PRS -1
<=>
malcoa_c C24H33N7O19P3S -5
ACP_c C11H21N2O7PRS -1
3ooctdACP_c C29H53N2O9PRS 0
co2_c CO2 0


In [81]:
met = model.metabolites.get_by_id('3ooctdACP_c')
met.charge = -1

In [82]:
foo.superprint(model,'3OAS60')

butACP_c + h_c + malACP_c --> 3ohexACP_c + ACP_c + co2_c 

malACP_c C14H22N2O10PRS -2
butACP_c C15H27N2O8PRS 0
h_c H 1
<=>
ACP_c C11H21N2O7PRS -1
3ohexACP_c C17H29N2O9PRS -1
co2_c CO2 0


In [83]:
met = model.metabolites.get_by_id('butACP_c')
met.charge = -1

In [84]:
foo.superprint(model,'3OAS80')

h_c + hexACP_c + malACP_c --> 3ooctACP_c + ACP_c + co2_c 

malACP_c C14H22N2O10PRS -2
hexACP_c C17H31N2O8PRS -1
h_c H 1
<=>
ACP_c C11H21N2O7PRS -1
3ooctACP_c C19H33N2O9PRS 0
co2_c CO2 0


In [85]:
met = model.metabolites.get_by_id('3ooctACP_c')
met.charge = -1

In [86]:
foo.superprint(model,'3OAR100')

3odecACP_c + h_c + nadph_c --> 3hdecACP_c + nadp_c 

nadph_c C21H26N7O17P3 -4
3odecACP_c C21H37N2O9PRS -1
h_c H 1
<=>
nadp_c C21H25N7O17P3 -3
3hdecACP_c C21H39N2O9PRS 0


In [87]:
met = model.metabolites.get_by_id('3hdecACP_c')
met.charge = -1

In [88]:
foo.superprint(model,'ACOATA')

ACP_c + accoa_c --> acACP_c + coa_c 

accoa_c C23H34N7O17P3S -4
ACP_c C11H21N2O7PRS -1
<=>
coa_c C21H32N7O16P3S -4
acACP_c C13H23N2O8PRS 0


In [89]:
met = model.metabolites.get_by_id('acACP_c')
met.charge = -1

In [90]:
foo.superprint(model,'RXN__45__9548')

h2o_c + ocdcaACP_c --> ACP_c + h_c + ocdca_c 

ocdcaACP_c C29H55N2O8PRS 0
h2o_c H2O 0
<=>
ACP_c C11H21N2O7PRS -1
h_c H 1
ocdca_c C18H35O2 -1


In [91]:
met = model.metabolites.get_by_id('ocdcaACP_c')
met.charge = -1

In [92]:
foo.superprint(model,'3OAR180')

3hoctaACP_c + nadp_c <=> 3ooctdACP_c + h_c + nadph_c 

nadp_c C21H25N7O17P3 -3
3hoctaACP_c C29H55N2O9PRS 0
<=>
3ooctdACP_c C29H53N2O9PRS -1
h_c H 1
nadph_c C21H26N7O17P3 -4


In [93]:
met = model.metabolites.get_by_id('3hoctaACP_c')
met.charge = -1

In [96]:
essential_reactions = find_essential_reactions(model)
unbalanced_list = foo.find_unbalanced_reactions(model)
check_and_fix = [x.id for x in essential_reactions if x.id in unbalanced_list[0]]
for x in check_and_fix:
    print x,unbalanced_list[0][x]

There are 298 unbalanced reactions and 0 metabolites with a faulty syntax
EAR140y {'C': 0, 'H': 0, 'charge': -1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
BIOMASS_REACTION {'C': -44.37737999999989, 'H': -128.8003400000002, 'charge': -44.89506000000003, 'O': -19.711309999999877, 'N': -5.5009199999999945, 'P': -0.9892799999999615, 'S': -0.082, 'R': 0}
COA1819ZD9DS {'C': 21.0, 'H': 27.0, 'charge': -4.0, 'O': 14.0, 'N': 7.0, 'P': 2.0, 'S': 0, 'R': 0}
3HAD180 {'C': 0, 'H': 0, 'charge': 1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
3OAR80_1 {'C': 0, 'H': 0, 'charge': 1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
KAS14 {'C': 0, 'H': 0, 'charge': 1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
EAR80y {'C': 0, 'H': 0, 'charge': -1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
EAR40y {'C': 0, 'H': 0, 'charge': -1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
PSD160 {'C': 0, 'H': 0, 'charge': -1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0, 'R': 0}
PSSA160 {'C': 0, 'H': 0, 'charge': 1.0, 'O': 0, 'N': 0, 'P': 0, 'S': 0

In [97]:
foo.superprint(model,'3HAD180')

3hoctaACP_c <=> h2o_c + toctd2eACP_c 

3hoctaACP_c C29H55N2O9PRS -1
<=>
h2o_c H2O 0
toctd2eACP_c C29H53N2O8PRS 0


In [98]:
model.metabolites.toctd2eACP_c.charge = -1

In [99]:
foo.superprint(model,'EAR140y')

h_c + nadph_c + tmrs2eACP_c --> myrsACP_c + nadp_c 

tmrs2eACP_c C25H45N2O8PRS 0
nadph_c C21H26N7O17P3 -4
h_c H 1
<=>
nadp_c C21H25N7O17P3 -3
myrsACP_c C25H47N2O8PRS -1


In [100]:
model.metabolites.tmrs2eACP_c.charge = -1

In [101]:
foo.superprint(model,'3OAR80_1')

3ooctACP_c + h_c + nadph_c --> 3hoctACP_c + nadp_c 

h_c H 1
3ooctACP_c C19H33N2O9PRS -1
nadph_c C21H26N7O17P3 -4
<=>
3hoctACP_c C19H35N2O9PRS 0
nadp_c C21H25N7O17P3 -3


In [102]:
model.metabolites.get_by_id('3hoctACP_c').charge = -1

In [103]:
foo.superprint(model,'3HAD80')

3hoctACP_c --> h2o_c + toct2eACP_c 

3hoctACP_c C19H35N2O9PRS -1
<=>
h2o_c H2O 0
toct2eACP_c C19H33N2O8PRS 0


In [104]:
model.metabolites.get_by_id('toct2eACP_c').charge = -1

In [105]:
foo.superprint(model,'PSSA160')

cdpdhdecg_c + ser__L_c --> cmp_c + h_c + ps160_c 

ser__L_c C3H7NO3 0
cdpdhdecg_c C44H79N3O15P2 -2
<=>
h_c H 1
cmp_c C9H12N3O8P -2
ps160_c C38H73N1O10P1 0


In [106]:
model.metabolites.get_by_id('ps160_c').charge = -1

In [107]:
foo.superprint(model,'KAS14')

acACP_c + h_c + malACP_c --> ACP_c + actACP_c + co2_c 

acACP_c C13H23N2O8PRS -1
malACP_c C14H22N2O10PRS -2
h_c H 1
<=>
ACP_c C11H21N2O7PRS -1
actACP_c C15H25N2O9PRS 0
co2_c CO2 0


In [108]:
model.metabolites.get_by_id('actACP_c').charge = -1

In [109]:
foo.superprint(model,'3OAR40')

actACP_c + h_c + nadph_c --> 3haACP_c + nadp_c 

h_c H 1
actACP_c C15H25N2O9PRS -1
nadph_c C21H26N7O17P3 -4
<=>
nadp_c C21H25N7O17P3 -3
3haACP_c C15H27N2O9PRS 0


In [110]:
model.metabolites.get_by_id('3haACP_c').charge = -1

In [111]:
foo.superprint(model,'3HAD140')

3hmrsACP_c --> h2o_c + tmrs2eACP_c 

3hmrsACP_c C25H47N2O9PRS 0
<=>
h2o_c H2O 0
tmrs2eACP_c C25H45N2O8PRS -1


In [112]:
model.metabolites.get_by_id('3hmrsACP_c').charge = -1

In [113]:
foo.superprint(model,'3OAR140')

3omrsACP_c + h_c + nadph_c --> 3hmrsACP_c + nadp_c 

nadph_c C21H26N7O17P3 -4
3omrsACP_c C25H45N2O9PRS 0
h_c H 1
<=>
nadp_c C21H25N7O17P3 -3
3hmrsACP_c C25H47N2O9PRS -1


In [114]:
model.metabolites.get_by_id('3omrsACP_c').charge = -1

In [115]:
foo.superprint(model,'3HAD40')

3haACP_c --> but2eACP_c + h2o_c 

3haACP_c C15H27N2O9PRS -1
<=>
but2eACP_c C15H25N2O8PRS 0
h2o_c H2O 0


In [116]:
model.metabolites.get_by_id('but2eACP_c').charge = -1

In [117]:
foo.superprint(model,'3OAS140')

ddcaACP_c + h_c + malACP_c <=> 3omrsACP_c + ACP_c + co2_c 

malACP_c C14H22N2O10PRS -2
ddcaACP_c C23H43N2O8PRS 0
h_c H 1
<=>
ACP_c C11H21N2O7PRS -1
3omrsACP_c C25H45N2O9PRS -1
co2_c CO2 0


In [118]:
model.metabolites.get_by_id('ddcaACP_c').charge = -1

In [119]:
foo.superprint(model,'3HAD100')

3hdecACP_c --> h2o_c + tdec2eACP_c 

3hdecACP_c C21H39N2O9PRS -1
<=>
h2o_c H2O 0
tdec2eACP_c C21H37N2O8PRS 0


In [120]:
model.metabolites.get_by_id('tdec2eACP_c').charge = -1

In [121]:
foo.superprint(model,'EAR100y')

h_c + nadph_c + tdec2eACP_c --> dcaACP_c + nadp_c 

tdec2eACP_c C21H37N2O8PRS -1
h_c H 1
nadph_c C21H26N7O17P3 -4
<=>
nadp_c C21H25N7O17P3 -3
dcaACP_c C21H39N2O8PRS 0


In [122]:
model.metabolites.get_by_id('dcaACP_c').charge = -1

In [123]:
foo.superprint(model,'3OAS120')

dcaACP_c + h_c + malACP_c --> 3oddecACP_c + ACP_c + co2_c 

malACP_c C14H22N2O10PRS -2
dcaACP_c C21H39N2O8PRS -1
h_c H 1
<=>
ACP_c C11H21N2O7PRS -1
co2_c CO2 0
3oddecACP_c C23H41N2O9PRS 0


In [124]:
model.metabolites.get_by_id('3oddecACP_c').charge = -1

In [125]:
foo.superprint(model,'EAR120y')

h_c + nadph_c + tddec2eACP_c --> ddcaACP_c + nadp_c 

tddec2eACP_c C23H41N2O8PRS 0
nadph_c C21H26N7O17P3 -4
h_c H 1
<=>
nadp_c C21H25N7O17P3 -3
ddcaACP_c C23H43N2O8PRS -1


In [126]:
model.metabolites.get_by_id('tddec2eACP_c').charge = -1

In [127]:
foo.superprint(model,'3OAR120')

3oddecACP_c + h_c + nadph_c --> 3hddecACP_c + nadp_c 

h_c H 1
nadph_c C21H26N7O17P3 -4
3oddecACP_c C23H41N2O9PRS -1
<=>
nadp_c C21H25N7O17P3 -3
3hddecACP_c C23H43N2O9PRS 0


In [128]:
model.metabolites.get_by_id('3hddecACP_c').charge = -1

In [129]:
foo.superprint(model,'COA1819ZD9DS')

focytb5_c + 2.0 h_c + o2_c + stcoa_c <=> ficytb5_c + 2.0 h2o_c + nadh_c + ocdce9coa_c 

focytb5_c None 0
stcoa_c C39H66N7O17P3S -4
o2_c O2 0
h_c H 1
<=>
nadh_c C21H27N7O14P2 -2
h2o_c H2O 0
ocdce9coa_c C39H64N7O17P3S -4
ficytb5_c None 0


In [130]:
model.reactions.COA1819ZD9DS.build_reaction_from_string('2.0 h_c + o2_c + stcoa_c + 2.0 focytb5_c <=> 2.0 h2o_c + 2.0 ficytb5_c + ocdce9coa_c')

In [131]:
model.reactions.COA1819ZD9DS.check_mass_balance()

{'charge': -2.0}

In [132]:
model.reactions.CYTOCHROME__45__B5__45__REDUCTASE__45__RXN.build_reaction_from_string('2.0 ficytb5_c + nadh_c <=> 2.0 focytb5_c + nad_c + h_c ')

In [133]:
model.metabolites.focytb5_c.formula = 'X'
model.metabolites.focytb5_c.charge = 0
model.metabolites.ficytb5_c.formula = 'X'
model.metabolites.ficytb5_c.charge = 1

In [134]:
foo.superprint(model,'CYTOCHROME__45__B5__45__REDUCTASE__45__RXN')

2.0 ficytb5_c + nadh_c <=> 2.0 focytb5_c + h_c + nad_c 

nadh_c C21H27N7O14P2 -2
ficytb5_c X 1
<=>
focytb5_c X 0
h_c H 1
nad_c C21H26N7O14P2 -1


In [135]:
foo.superprint(model,'COA1819ZD9DS')

2.0 focytb5_c + 2.0 h_c + o2_c + stcoa_c <=> 2.0 ficytb5_c + 2.0 h2o_c + ocdce9coa_c 

stcoa_c C39H66N7O17P3S -4
focytb5_c X 0
h_c H 1
o2_c O2 0
<=>
ocdce9coa_c C39H64N7O17P3S -4
h2o_c H2O 0
ficytb5_c X 1


In [136]:
foo.superprint(model,'_1__46__14__46__19__46__6__45__RXN')

h_c + nadph_c + ocdce9coa_c --> h2o_c + lnlccoa_c + nadp_c 

ocdce9coa_c C39H64N7O17P3S -4
h_c H 1
nadph_c C21H26N7O17P3 -4
<=>
nadp_c C21H25N7O17P3 -3
lnlccoa_c C39H62N7O17P3S -4
h2o_c H2O 0


In [137]:
model.reactions._1__46__14__46__19__46__6__45__RXN.build_reaction_from_string('2.0 focytb5_c + 2.0 h_c + o2_c + ocdce9coa_c -> 2.0 ficytb5_c + 2.0 h2o_c + lnlccoa_c')

In [138]:
essential_reactions = find_essential_reactions(model)
unbalanced_list = foo.find_unbalanced_reactions(model)
check_and_fix = [x.id for x in essential_reactions if x.id in unbalanced_list[0]]
for x in check_and_fix:
    print x,unbalanced_list[0][x]

There are 284 unbalanced reactions and 0 metabolites with a faulty syntax
BIOMASS_REACTION {'C': -44.37737999999989, 'H': -128.8003400000002, 'charge': -44.89506000000003, 'O': -19.711309999999877, 'N': -5.5009199999999945, 'P': -0.9892799999999615, 'S': -0.082, 'R': 0}


In [139]:
unbalanced_list = foo.find_unbalanced_reactions(model)

There are 284 unbalanced reactions and 0 metabolites with a faulty syntax


In [140]:
model.remove_reactions(['ACP_REPLENISHMENT_RXN'],remove_orphans=True)

In [142]:
model.optimize().f

0.9879687633916031

In [143]:
for rxn in model.reactions:
    if rxn.id.startswith('EX_') and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_o2_e: o2_e <=>  -1000.0 1000.0 -6.34703242588
EX_co2_e: co2_e <=>  -1000.0 1000.0 -5.9173895099
EX_nh4_e: nh4_e <=>  -1000.0 1000.0 -25.4799614001
EX_pi_e: pi_e <=>  -1000.0 1000.0 -0.977377738248
EX_so4_e: so4_e <=>  -1000.0 1000.0 -0.0810134385981
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 47.2143257447
EX_nac_e: nac_e <=>  -1000.0 1000.0 -3.5527136788e-14
EX_glc__D_e: glc__D_e <=>  -20 1000 -20.0
EX_fum_e: fum_e -->  0.0 1000.0 0.47325679704
EX_asp__L_e: asp__L_e -->  0.0 1000.0 20.0452242701


In [145]:
model.reactions.EX_nac_e.bounds= 0, 0

In [146]:
model.optimize().f

0.9879687633916031

In [149]:
model.reactions.EX_glc__D_e.bounds=0, 0

In [150]:
model.optimize().f

-5.967066432441049e-15

In [154]:
model.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x01141e3290
Stoichiometry,glc__D_e --> None -->
GPR,
Lower bound,0
Upper bound,0


In [155]:
model.reactions.EX_glc__D_e.bounds= -20, 0

In [156]:
model.optimize().f

0.9879687633916031

In [157]:
print model.objective

Maximize
-1.0*BIOMASS_REACTION_reverse_42726 + 1.0*BIOMASS_REACTION


### Find blocked reactions

In [164]:
from cobra.flux_analysis.variability import find_blocked_reactions

In [166]:
result = find_blocked_reactions(model)

In [169]:
fully_blocked = [x for x in result if not x.startswith('TRANS__45')]

In [170]:
from cameo.flux_analysis import structural

In [171]:
dead_ends = structural.find_dead_end_reactions(model)

In [172]:
len(dead_ends)

740

In [173]:
dead_ends_non_transport = [x for x in dead_ends if not x.startswith('TRANS__45')]

In [174]:
len(dead_ends_non_transport)

648

In [176]:
non_dead_ends_unbalanced = []

for rx in model.reactions:
    if rx.id not in dead_ends_non_transport:
        if rx.id not in result:
            marker = False
            for met in rx.metabolites:
                if met.formula is None:
                    marker = True
            if marker == True:
                non_dead_ends_unbalanced.append(rx.id)

In [177]:
len(non_dead_ends_unbalanced)

49

In [178]:
non_dead_ends_metabolites = []
for rxn_id in non_dead_ends_unbalanced:
    for met in model.reactions.get_by_id(rxn_id).metabolites:
        if met.formula is None and met.id not in non_dead_ends_metabolites:
            non_dead_ends_metabolites.append(met.id)

In [179]:
non_dead_ends_unbalanced

['_1__46__2__46__4__46__4__45__RXN',
 'RXN__45__7719',
 '_2__46__3__46__1__46__168__45__RXN',
 'RXN__45__5781',
 'PHOSPHATIDYLINOSITOL__45__BISPHOSPHATASE__45__RXN',
 'RXN__45__15212',
 '_1__45__PHOSPHATIDYLINOSITOL__45__3__45__KINASE__45__RXN',
 'CITRYLY__45__RXN',
 'CITTRANS__45__RXN',
 'BUTYRYL__45__COA__45__DEHYDROGENASE__45__RXN',
 'RXN__45__12959',
 'PHOSPHATIDATE__45__PHOSPHATASE__45__RXN',
 'CARBONYL__45__REDUCTASE__45__NADPH__45__RXN',
 'RXN__45__10938',
 '_2__46__7__46__1__46__150__45__RXN',
 '_2__46__7__46__1__46__68__45__RXN',
 'RXN0__45__5055',
 'PHOSPHOLIPASE__45__C__45__RXN',
 '_2__46__7__46__1__46__149__45__RXN',
 'RXN__45__12460',
 'BIOTIN__45__CARBOXYL__45__RXN',
 'PHOSPHATIDYLINOSITOL__45__3__45__PHOSPHATASE__45__RXN',
 'ASPARAGINE__45____45__TRNA__45__LIGASE__45__RXN',
 'RXN__45__5061',
 'RXN__45__15211',
 'PYRUFLAVREDUCT__45__RXN',
 'PROTEIN__45__KINASE__45__RXN',
 'RXN__45__13417',
 'RXN__45__13416',
 'RXN__45__13415',
 '_2__46__5__46__1__46__46__45__RXN',
 '_1__4

### Removal of any reactions using generic metabolites

In [180]:
generic_rxns_ids = [x.id for x in model.metabolites.Carboxylates_c.reactions]
model.remove_reactions(generic_rxns_ids,remove_orphans=True)

In [181]:
generic_rxns_ids = [x.id for x in model.metabolites.Amino__45__Acids_c.reactions]
model.remove_reactions(generic_rxns_ids,remove_orphans=True)

In [182]:
generic_rxns_ids = [x.id for x in model.metabolites.N__45__Carbamoyl__45__L__45__Amino__45__Acids_c.reactions]
model.remove_reactions(generic_rxns_ids,remove_orphans=True)

In [183]:
generic_rxns_ids = [x.id for x in model.metabolites.L__45__2__45__AMINO__45__ACID_c.reactions]
model.remove_reactions(generic_rxns_ids,remove_orphans=True)

In [184]:
generic_rxns_ids = [x.id for x in model.metabolites.Amino__45__Acids__45__20_c.reactions]
model.remove_reactions(generic_rxns_ids,remove_orphans=True)

### Sphingosine/ Cermide conversions knocked out, but left in the model for later review

In [185]:
model.reactions.RXN__45__15212.knock_out()
model.reactions.RXN__45__12460.knock_out()

### Use the balanced respiratory chain from the iMM904 S. cerevisiae model

In [186]:
met = model.metabolites.focytc_m
met.formula = 'C42H53FeN8O6S2'
met.charge = 1

In [187]:
met = model.metabolites.ficytc_m
met.formula = 'C42H52FeN8O6S2'
met.charge = 1

In [188]:
model.reactions.CYOOm_umay.build_reaction_from_string(
    '4.0 focytc_m + 6.0 h_m + o2_m -> 6.0 h_p + 4.0 ficytc_m + 2.0 h2o_m')

In [189]:
model.reactions.CYOR_u6m_umay.build_reaction_from_string(
'2.0 ficytc_m + 1.5 h_m + q6h2_m -> 1.5 h_p + 2.0 focytc_m + q6_m')

### Phosphatidyl Inositol Transformations

In [190]:
met = model.metabolites.L__45__1__45__ehosphatidyl__45__inositols_c
met.name = 'Phosphatidyl 1D myo inositol'
met.formula = 'C41H78O13P'
met.charge = 0
met.id = 'ptd1ino160_c'

In [191]:
rxn = model.reactions._2__46__7__46__8__46__11__45__RXN
rxn.name = 'CDP diacylglycerol-inositol 3-phosphatidyltransferase'
rxn.id = 'CDPPT160'

In [192]:
# I decided to delete all the non-metabolic kinase reactions of phosphatidyl inositol
reactions = ['_1__45__PHOSPHATIDYLINOSITOL__45__3__45__KINASE__45__RXN',
 'RXN__45__10938','_2__46__7__46__1__46__150__45__RXN',
            'RXN__45__10958','_2__46__7__46__1__46__149__45__RXN',
            'PHOSPHATIDYLINOSITOL__45__BISPHOSPHATASE__45__RXN','_1__45__PHOSPHATIDYLINOSITOL__45__KINASE__45__RXN',
            'PHOSPHATIDYLINOSITOL__45__3__45__PHOSPHATASE__45__RXN','_2__46__7__46__1__46__153__45__RXN',
            '_2__46__7__46__1__46__68__45__RXN',
            '_2__46__7__46__1__46__154__45__RXN',
            'RXN__45__9779',
            '_3__46__1__46__4__46__11__45__RXN',
             '_3__46__1__46__4__46__10__45__RXN',
            'RXN__45__13334',
            '_2__46__4__46__1__46__198__45__RXN']
model.remove_reactions(reactions,remove_orphans=True)

### Removal of generic 'Nucleosides' Transport reactions

In [193]:
reactions = ['TRANS__45__RXN0__45__468',
 'TRANS__45__RXNTFS__45__24']
model.remove_reactions(reactions,remove_orphans=True)

### Removal of Biotin Carrier Biosynthesis and Reaction cycle

In [194]:
reactions = ['BIOTINLIG__45__RXN','RXN__45__7101','RXN0__45__5055',
 'BIOTIN__45__CARBOXYL__45__RXN']
model.remove_reactions(reactions,remove_orphans=True)

In [195]:
model.reactions.ACCOAC.gene_reaction_rule = 'UMAG_04629 or UMAG_05131 or UMAG_11593 or UMAG_03839'

### Removal of generic Carbonyl-Reductase (NADPH) and Alcohol-Oxidase (NAD) Reactions

In [196]:
reactions = ['RXN__45__12448',
 'CARBONYL__45__REDUCTASE__45__NADPH__45__RXN']
model.remove_reactions(reactions,remove_orphans=True)

### Removal of non-metabolic histone methyl/ acetyl transferase reactions

In [197]:
reactions = ['HISTONE__45__ACETYLTRANSFERASE__45__RXN',
 '_3__46__5__46__1__46__98__45__RXN']
model.remove_reactions(reactions,remove_orphans=True)

### Removal of non-metabolic protein kinase reactions

In [198]:
reactions = ['PROTEIN__45__KINASE__45__RXN',
 '_3__46__1__46__3__46__16__45__RXN',
            'RXN__45__9920',
            '_2__46__7__46__12__46__1__45__RXN']
model.remove_reactions(reactions,remove_orphans=True)

### Consensus definition of Ferredoxins

In [199]:
model.metabolites.Oxidized__45__ferredoxins_c.reactions

frozenset({<Reaction RXN__45__5061 at 0x113db4850>,
           <Reaction PYRUFLAVREDUCT__45__RXN at 0x113dee190>,
           <Reaction _1__46__18__46__1__46__2__45__RXN at 0x113e26110>,
           <Reaction FERREDOXIN__45____45__NITRITE__45__REDUCTASE__45__RXN at 0x113fb10d0>})

In [200]:
met = model.metabolites.Oxidized__45__ferredoxins_c
met.formula = 'X'
met.charge = 0
met.name = 'Oxidized ferredoxin'
met.id = 'fdxox_c'

In [201]:
met = model.metabolites.fdxrd_c
met.formula = 'XH2'
met.charge = 0
met.name = 'Reduced ferredoxin'

### Removal of Archeal Lysine Biosynthethis Pathway

In [202]:
reactions = ['RXN__45__15004',
 'RXN__45__15007',
 'RXN__45__15006',
 'RXN__45__15005',
 'RXN__45__15003']

In [203]:
model.remove_reactions(reactions,remove_orphans=True)

### Removal of non-metabolic Deoxyhypusine Biosynthesis (post-translational modification catalyzed by UMAG_00071)

In [204]:
reactions = ['RXN__45__13417','RXN__45__13416','RXN__45__13415','RXN__45__13414','_2__46__5__46__1__46__46__45__RXN']

In [205]:
model.remove_reactions(reactions,remove_orphans=True)

### Replacement of all ETF metabolites with the fad/fadh2 metabolites used in the model

In [206]:
foo.swap_mets(model.metabolites.ETF__45__Oxidized_c,model.metabolites.fadh2_c)

In [207]:
foo.swap_mets(model.metabolites.ETF__45__Reduced_c,model.metabolites.fad_c)

In [208]:
model.metabolites.remove('ETF__45__Oxidized_c')

In [209]:
model.metabolites.remove('ETF__45__Reduced_c')

### Removal of Citrate Lyase Bypass through 2 subreactions

In [210]:
cit_ly_rxns = ['CITTRANS__45__RXN','CITRYLY__45__RXN']

In [211]:
model.remove_reactions(cit_ly_rxns,remove_orphans=True)

### Removal of 3 subreactions of  2-oxoisovalerate dehydrogenase

In [212]:
OIVD2m_instances = ['_1__46__2__46__4__46__4__45__RXN','RXN__45__7719','_2__46__3__46__1__46__168__45__RXN']

In [213]:
merge_set = set()
for rxn_id in OIVD2m_instances:
        merge_set.update(model.reactions.get_by_id(rxn_id).genes)

In [214]:
gpr_str = ''
for gene in list(merge_set):
    if gpr_str == '':
        gpr_str = gene.id
    else:
        gpr_str += ' or ' + gene.id

In [215]:
model.reactions.OIVD2m.gene_reaction_rule = gpr_str

In [216]:
model.remove_reactions(OIVD2m_instances,remove_orphans=True)

### Diacylglycerol Transformations

In [217]:
foo.swap_mets(model.metabolites.L__45__PHOSPHATIDATE_c,model.metabolites.pa160_c)

In [218]:
model.metabolites.remove('L__45__PHOSPHATIDATE_c')

In [219]:
model.metabolites.pa160_c.reactions

frozenset({<Reaction PHOSCHOL__45__RXN__45__1__45__2__45__DIPALMITOYLPHOSPHATIDYLCHOLINE__47__WATER__47____47__CPD0__45__1422__47__CHOLINE__47__PROTON__46__67__46__ at 0x113a8b1d0>,
           <Reaction RXN__45__12959 at 0x113ba9fd0>,
           <Reaction PHOSPHATIDATE__45__PHOSPHATASE__45__RXN at 0x113bbc290>,
           <Reaction RXN__45__1623__45__PALMITYL__45__COA__47__1__45__PALMITOYLGLYCEROL__45__3__45__PHOSPHATE__47____47__CPD0__45__1422__47__CO__45__A__46__61__46__ at 0x113bbc310>,
           <Reaction DASYN160 at 0x113bbc5d0>,
           <Reaction AGPAT160 at 0x113bf3410>,
           <Reaction RXN0__45__6705 at 0x113bf3810>})

In [220]:
rxn = model.reactions.PHOSPHATIDATE__45__PHOSPHATASE__45__RXN
rxn.name = 'Phosphatidate phosphatase (n-C16:0)'
rxn.id = 'PAPA160'

In [221]:
model.reactions.PAPA160

Reaction identifier,PAPA160
Name,Phosphatidate phosphatase (n-C16:0)
Memory address,0x0113bbc290
Stoichiometry,"h2o_c + pa160_c --> 12dgr_EC_c + pi_c H2O + dipalmitoyl phosphatidate --> a 1,2-diacyl-sn-glycerol + phosphate"
GPR,UMAG_10421
Lower bound,0.0
Upper bound,1000.0


In [222]:
met = model.metabolites.get_by_id('12dgr_EC_c')
met.formula = 'C35H68O5'
met.charge = 0
met.name = '1,2-Diacyl-sn-glycerol (dihexadecanoyl, n-C16:0)'
met.id = '12dgr160_c'

### Remove orphans with no reactions at all

In [223]:
orphans = []
for met in model.metabolites:
    orphan = foo.is_orphan(model,met)
    if orphan:
        orphans.append(met.id)

orphans_c = [x for x in orphans if x.endswith('_c')]

In [224]:
len(orphans_c)

182

In [225]:
for met in orphans_c:
    if not model.metabolites.get_by_id(met).reactions:
        model.metabolites.remove(met)
        orphans_c.remove(met)

In [226]:
orphans = []
for met in model.metabolites:
    orphan = foo.is_orphan(model,met)
    if orphan:
        orphans.append(met.id)

orphans_c = [x for x in orphans if x.endswith('_c')]

In [227]:
len(orphans_c)

170

### Find orphans that are consumed in reactions that have no GPR

In [228]:
no_gene_deadend_rxns = []

for met in orphans_c:
    if model.metabolites.get_by_id(met).reactions:
        for rxn in model.metabolites.get_by_id(met).reactions:
            if not rxn.genes:
                no_gene_deadend_rxns.append(rxn.id)

### Final Balancing of those reactions with GPR

In [229]:
unbalanced_list = foo.find_unbalanced_reactions(model)

There are 234 unbalanced reactions and 0 metabolites with a faulty syntax


In [232]:
unbalanced_list_GPR = []

for rxn in unbalanced_list[0].keys():
    if rxn not in result and model.reactions.get_by_id(rxn).genes:
        unbalanced_list_GPR.append(rxn)

In [233]:
for idx in unbalanced_list_GPR:
    foo.superprint(model,idx)
    print '-----------'*7

CPD__45__9965_c + o2_c --> POLYMER__45__INST__45__TRANS__45__D2__45__ENOYL__45__COA__45__C16__45__H32_c + h2o2_c 

o2_c O2 0
CPD__45__9965_c C41H70N7O17P3S -4
<=>
POLYMER__45__INST__45__TRANS__45__D2__45__ENOYL__45__COA__45__C16__45__H32_c C41H68N7O17P3S None
h2o2_c H2O2 0
-----------------------------------------------------------------------------
dd2coa_c + h2o_c --> POLYMER__45__INST__45__D__45__3__45__HYDROXYACYL__45__COA__45__C8__45__H16_c 

dd2coa_c C33H52N7O17P3S -4
h2o_c H2O 0
<=>
POLYMER__45__INST__45__D__45__3__45__HYDROXYACYL__45__COA__45__C8__45__H16_c C33H54N7O18P3S None
-----------------------------------------------------------------------------
CPD__45__10261_c + nad_c --> POLYMER__45__INST__45__3__45__KETOACYL__45__COA__45__C14__45__H28_c + h_c + nadh_c 

CPD__45__10261_c C39H66N7O18P3S -4
nad_c C21H26N7O14P2 -1
<=>
POLYMER__45__INST__45__3__45__KETOACYL__45__COA__45__C14__45__H28_c C39H64N7O18P3S None
nadh_c C21H27N7O14P2 -2
h_c H 1
----------------------------------

In [242]:
for i, rxn in enumerate(non_dead_ends_unbalanced):
    if rxn in model.reactions:
        foo.superprint(model,rxn)
        print '--'*50
        print rxn
    else:
        non_dead_ends_unbalanced.pop(i)

12dgr160_c + ctp_c --> cdp_c + h_c + pa160_c 

12dgr160_c C35H68O5 0
ctp_c C9H12N3O14P3 -4
<=>
pa160_c C35H67O8P -2
cdp_c C9H12N3O11P2 -3
h_c H 1
----------------------------------------------------------------------------------------------------
RXN__45__12959
h2o_c + pc160_c --> 12dgr160_c + cholp_c + h_c 

pc160_c C40H80N1O8P1 0
h2o_c H2O 0
<=>
cholp_c C5H13NO4P -1
12dgr160_c C35H68O5 0
h_c H 1
----------------------------------------------------------------------------------------------------
PHOSPHOLIPASE__45__C__45__RXN
asn__L_c + atp_c + h_c + trnaasn_c --> Charged__45__ASN__45__tRNAs_c + amp_c + ppi_c 

trnaasn_c C10H17O10PR2 0
asn__L_c C4H8N2O3 0
atp_c C10H12N5O13P3 -4
h_c H 1
<=>
Charged__45__ASN__45__tRNAs_c None 1
amp_c C10H12N5O7P -2
ppi_c HO7P2 -3
----------------------------------------------------------------------------------------------------
ASPARAGINE__45____45__TRNA__45__LIGASE__45__RXN
2.0 fdxrd_c + 2.0 h_c + mlthf_c --> 5mthf_c + 2.0 fdxox_c 

mlthf_c C20H21N7O6 

### Addition of Cyp3 - P450 monooxygenase Umag_05074

In [243]:
model.metabolites.itacon_c.formula

'C5H4O4'

In [244]:
met = Metabolite(id='2hp_c',formula='C5H5O5',name='2-Hydroxyparaconate',compartment='c')

In [245]:
met.charge = -1

In [246]:
rxn = Reaction(id='IMO',name='Itaconate monooxigenase - Cyp3 P450 Monooxygenase')

In [247]:
rxn.add_metabolites({met:1,
                    model.metabolites.itacon_c:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadp_c:1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-2,
                    model.metabolites.h2o_c:1})

In [248]:
rxn.check_mass_balance()

{}

In [249]:
rxn.gene_reaction_rule = 'UMAG_05074'

In [250]:
model.add_reaction(rxn)

In [253]:
model.add_boundary(model.metabolites.get_by_id('2hp_c'),type='demand')
model.reactions.DM_2hp_c.bounds= 0,1000

In [255]:
model.optimize().f

0.9879687633915931

In [256]:
for rxn in model.reactions:
    if rxn.id.startswith('EX_') and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_o2_e: o2_e <=>  -1000.0 1000.0 -6.34703242588
EX_co2_e: co2_e <=>  -1000.0 1000.0 -5.9173895099
EX_nh4_e: nh4_e <=>  -1000.0 1000.0 -5.92696292721
EX_pi_e: pi_e <=>  -1000.0 1000.0 -0.977377738248
EX_so4_e: so4_e <=>  -1000.0 1000.0 -0.0810134385981
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 27.4152143732
EX_glc__D_e: glc__D_e <--  -20 0 -20.0
EX_mal__L_e: mal__L_e -->  0.0 1000.0 19.7991113715
EX_fum_e: fum_e -->  0.0 1000.0 0.47325679704
EX_gly_e: gly_e -->  0.0 1000.0 0.492225797297


In [258]:
model.objective = model.reactions.DM_2hp_c

In [259]:
print model.objective

Maximize
1.0*DM_2hp_c - 1.0*DM_2hp_c_reverse_2d6fc


In [260]:
model.reactions.EX_glc__D_e.bounds = -10, 1000

In [261]:
model.optimize().f

9.999999999992852

In [262]:
for rxn in model.reactions:
    if (rxn.id.startswith('EX_') or rxn.id.startswith('DM_')) and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_o2_e: o2_e <=>  -1000.0 1000.0 -24.8484848485
EX_co2_e: co2_e <=>  -1000.0 1000.0 9.99999999993
EX_pi_e: pi_e <=>  -1000.0 1000.0 3.45578302299e-11
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 39.6969696968
EX_na1_e: na1_e <=>  -1000.0 1000.0 3.23117426779e-27
EX_glc__D_e: glc__D_e <=>  -10 1000 -10.0
EX_fum_e: fum_e -->  0.0 1000.0 -2.30374701798e-11
EX_asn__L_e: asn__L_e -->  0.0 1000.0 4.62236322155e-11
EX_cys__L_e: cys__L_e -->  0.0 1000.0 1.65524167187e-13
EX_gly_e: gly_e -->  0.0 1000.0 -1.25990320828e-11
DM_2hp_c: 2hp_c -->  0 1000 9.99999999999


# Test carbon sources that umay should be able to grow on

In [279]:
model.objective = model.reactions.BIOMASS_REACTION

In [282]:
model.reactions.EX_glc__D_e.bounds= 0, 1000

In [283]:
model.optimize().f

0.0

In [284]:
model.add_boundary(model.metabolites.salc_e,type='exchange')


ValueError: boundary EX_salc_e already exists

In [267]:
model.reactions.EX_salc_e.bounds= -10, 1000

In [268]:
model.optimize().f

0.0

In [269]:
model.reactions.EX_salc_e.bounds= 0, 1000

In [270]:
model.metabolites.get_by_id('4hbz_c').reactions

frozenset({<Reaction BENZOATE__45__4__45__MONOOXYGENASE__45__RXN at 0x1132b7410>})

In [271]:
model.metabolites.get_by_id('5dglcn_c').reactions

frozenset({<Reaction GLUCONATE__45__5__45__DEHYDROGENASE__45__RXN__45__GLUCONATE__47__NAD__47____47__5__45__DEHYDROGLUCONATE__47__NADH__47__PROTON__46__46__46__ at 0x113bbcc10>,
           <Reaction GLUCONATE__45__5__45__DEHYDROGENASE__45__RXN__45__GLUCONATE__47__NADP__47____47__5__45__DEHYDROGLUCONATE__47__NADPH__47__PROTON__46__48__46__ at 0x113bbcc90>})

In [272]:
model.add_boundary(model.metabolites.get_by_id('5dglcn_c'),type='demand')

Reaction identifier,DM_5dglcn_c
Name,5-dehydro-D-gluconate demand
Memory address,0x0113e26b50
Stoichiometry,5dglcn_c --> 5-dehydro-D-gluconate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [273]:
model.reactions.DM_5dglcn_c.bounds= -10, 1000

In [274]:
foo.invert_reaction(model.reactions.GLUCONATE__45__5__45__DEHYDROGENASE__45__RXN__45__GLUCONATE__47__NADP__47____47__5__45__DEHYDROGLUCONATE__47__NADPH__47__PROTON__46__48__46__)

In [275]:
foo.invert_reaction(model.reactions.GLUCONATE__45__5__45__DEHYDROGENASE__45__RXN__45__GLUCONATE__47__NAD__47____47__5__45__DEHYDROGLUCONATE__47__NADH__47__PROTON__46__46__46__)

In [276]:
model.optimize().f

9.999999999999988

In [280]:
model.reactions.DM_5dglcn_c.bounds= 0, 1000

In [281]:
model.optimize().f

0.0

In [285]:
model.add_boundary(model.metabolites.get_by_id('ac_e'),type='exchange')

Reaction identifier,EX_ac_e
Name,acetate exchange
Memory address,0x0113875510
Stoichiometry,ac_e <=> acetate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [286]:
model.reactions.EX_ac_e.bounds= -1000, 1000

In [287]:
model.metabolites.ac_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__58__45__ACET__47__PROTON__47____47__ACET__47__PROTON__46__25__46__ at 0x113422390>,
           <Reaction EX_ac_e at 0x113875510>,
           <Reaction TRANS__45__RXNTFS__45__7__45__ACET__47____47__ACET__46__11__46__ at 0x113ed2290>,
           <Reaction TRANS__45__RXNTFS__45__8__45__ACET__47____47__ACET__46__11__46__ at 0x113efc310>,
           <Reaction TRANS__45__RXNTFS__45__1__45__ACET__47____47__ACET__46__11__46__ at 0x113fdd410>})

In [288]:
model.metabolites.ac_c.reactions

frozenset({<Reaction RXN__45__14728 at 0x1132b78d0>,
           <Reaction TRANS__45__RXNTFS__45__58__45__ACET__47__PROTON__47____47__ACET__47__PROTON__46__25__46__ at 0x113422390>,
           <Reaction ACSp at 0x113a64d50>,
           <Reaction ACETATE__45____45__COA__45__LIGASE__45__ADP__45__FORMING__45__RXN at 0x113a77190>,
           <Reaction LO at 0x113b82090>,
           <Reaction _3__46__1__46__1__46__56__45__RXN at 0x113c41690>,
           <Reaction AHSERL at 0x113cb4210>,
           <Reaction AHSERL2 at 0x113cb4290>,
           <Reaction _3__46__1__46__1__46__69__45__RXN at 0x113cf6dd0>,
           <Reaction _3__46__1__46__1__46__47__45__RXN at 0x113d09cd0>,
           <Reaction ACKrm at 0x113da2710>,
           <Reaction METB1 at 0x113dda8d0>,
           <Reaction TRANS__45__RXNTFS__45__7__45__ACET__47____47__ACET__46__11__46__ at 0x113ed2290>,
           <Reaction TRANS__45__RXNTFS__45__8__45__ACET__47____47__ACET__46__11__46__ at 0x113efc310>,
           <Reaction TRANS__45

In [289]:
model.optimize().f

0.0

In [459]:
model.reactions.EX_ac_e.bounds= 0, 1000

In [300]:
model.metabolites.get_by_id('2obut_e').reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__7__45__2__45__OXOBUTANOATE__47____47__2__45__OXOBUTANOATE__46__31__46__ at 0x113e98ad0>,
           <Reaction TRANS__45__RXNTFS__45__1__45__2__45__OXOBUTANOATE__47____47__2__45__OXOBUTANOATE__46__31__46__ at 0x113fb1f50>})

In [301]:
model.add_boundary(model.metabolites.get_by_id('2obut_e'),type="exchange")

Reaction identifier,EX_2obut_e
Name,2-oxobutanoate exchange
Memory address,0x0119152d90
Stoichiometry,2obut_e <=> 2-oxobutanoate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [302]:
model.reactions.EX_2obut_e.bounds=-10, 1000

In [303]:
model.optimize().f

0.14450196746653718

In [304]:
model.reactions.EX_2obut_e.bounds= 0, 1000

In [305]:
model.metabolites.akg_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__7__45__2__45__KETOGLUTARATE__47____47__2__45__KETOGLUTARATE__46__33__46__ at 0x113e98910>,
           <Reaction TRANS__45__RXNTFS__45__1__45__2__45__KETOGLUTARATE__47____47__2__45__KETOGLUTARATE__46__33__46__ at 0x113fb1b10>})

In [306]:
model.add_boundary(model.metabolites.akg_e, type='exchange')

Reaction identifier,EX_akg_e
Name,2-oxoglutarate exchange
Memory address,0x01191520d0
Stoichiometry,akg_e <=> 2-oxoglutarate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [307]:
model.reactions.EX_akg_e.bounds= -1000, 1000

In [308]:
model.optimize().f

1.947743101374853

In [309]:
model.reactions.EX_akg_e.bounds= 0, 1000

In [310]:
model.metabolites.but_c.reactions

frozenset({<Reaction ACYLCOASYN__45__RXN__45__BUTYRIC_ACID__47__CO__45__A__47__ATP__47____47__BUTYRYL__45__COA__47__PPI__47__AMP__46__43__46__ at 0x113b0d650>,
           <Reaction BUTYRATE__45____45__COA__45__LIGASE__45__RXN__45__BUTYRIC_ACID__47__ATP__47__CO__45__A__47____47__BUTYRYL__45__COA__47__AMP__47__PPI__46__43__46__ at 0x113b33210>,
           <Reaction RXN__45__12086 at 0x113c8ec50>,
           <Reaction TRANS__45__RXNTFS__45__7__45__BUTYRIC_ACID__47____47__BUTYRIC_ACID__46__27__46__ at 0x113eaa550>,
           <Reaction TRANS__45__RXNTFS__45__1__45__BUTYRIC_ACID__47____47__BUTYRIC_ACID__46__27__46__ at 0x113fc9510>,
           <Reaction ACYL__45__COA__45__HYDROLASE__45__RXN__45__BUTYRYL__45__COA__47__WATER__47____47__BUTYRIC_ACID__47__CO__45__A__47__PROTON__46__44__46__ at 0x1140c5490>})

In [311]:
model.add_boundary(model.metabolites.but_e,type='exchange')
model.reactions.EX_but_e.bounds= -1000, 1000
model.optimize().f

3.4663348173457567

In [312]:
model.reactions.EX_but_e.bounds= 0, 1000

In [313]:
model.optimize().f

4.304739546668171e-15

In [314]:
model.reactions.EX_mal__L_e.bounds= -1000, 1000
model.optimize().f

4.304739546668171e-15

In [315]:
model.metabolites.mal__L_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__7__45__MAL__47____47__MAL__46__9__46__ at 0x113ebf390>,
           <Reaction TRANS__45__RXNTFS__45__1__45__MAL__47____47__MAL__46__9__46__ at 0x113fc9b90>,
           <Reaction TRANS__45__RXNTFS__45__53__45__MAL__47____47__MAL__46__9__46__ at 0x11419bfd0>,
           <Reaction EX_mal__L_e at 0x1141e3390>})

In [316]:
model.reactions.TRANS__45__RXNTFS__45__53__45__MAL__47____47__MAL__46__9__46__.lower_bound = -1000

In [317]:
model.optimize().f

4.304739546668171e-15

In [318]:
model.reactions.EX_mal__L_e.bounds= 0, 1000

In [319]:
model.metabolites.ala__L_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__16__45__L__45__ALPHA__45__ALANINE__47__PROTON__47____47__L__45__ALPHA__45__ALANINE__47__PROTON__46__47__46__ at 0x113f3ee10>,
           <Reaction TRANS__45__RXNTFS__45__26__45__L__45__ALPHA__45__ALANINE__47____47__L__45__ALPHA__45__ALANINE__46__33__46__ at 0x113f96390>,
           <Reaction TRANS__45__RXNTFS__45__33__45__L__45__ALPHA__45__ALANINE__47__PROTON__47____47__L__45__ALPHA__45__ALANINE__47__PROTON__46__47__46__ at 0x114039dd0>,
           <Reaction EX_ala__L_e at 0x1141e34d0>})

In [320]:
for rxn in model.metabolites.ala__L_e.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__RXNTFS__45__16__45__L__45__ALPHA__45__ALANINE__47__PROTON__47____47__L__45__ALPHA__45__ALANINE__47__PROTON__46__47__46__: ala__L_e + h_e --> ala__L_c + h_c 0.0 1000.0
TRANS__45__RXNTFS__45__26__45__L__45__ALPHA__45__ALANINE__47____47__L__45__ALPHA__45__ALANINE__46__33__46__: ala__L_e <=> ala__L_c -1000.0 1000.0
TRANS__45__RXNTFS__45__33__45__L__45__ALPHA__45__ALANINE__47__PROTON__47____47__L__45__ALPHA__45__ALANINE__47__PROTON__46__47__46__: ala__L_e + h_e --> ala__L_c + h_c 0.0 1000.0
EX_ala__L_e: ala__L_e -->  0.0 1000.0


In [321]:
model.reactions.EX_ala__L_e.bounds= -1000, 1000
model.optimize().f

3.2520405031916395

In [322]:
model.reactions.EX_ala__L_e.bounds= 0, 1000

In [323]:
for rxn in model.metabolites.ala_B_e.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__RXNTFS__45__33__45__B__45__ALANINE__47__PROTON__47____47__B__45__ALANINE__47__PROTON__46__35__46__: ala_B_e + h_e --> ala_B_c + h_c 0.0 1000.0


In [324]:
model.add_boundary(model.metabolites.ala_B_e,type='exchange')
model.reactions.EX_ala_B_e.bounds= -1000, 1000
model.optimize().f

9.298401885666734e-14

In [325]:
model.reactions.TRANS__45__RXNTFS__45__33__45__B__45__ALANINE__47__PROTON__47____47__B__45__ALANINE__47__PROTON__46__35__46__.lower_bound = -1000

In [326]:
model.optimize().f

9.298401885666734e-14

In [327]:
model.reactions.EX_ala_B_e.bounds= 0, 1000

In [328]:
model.metabolites.arab__L_c

Metabolite identifier,arab__L_c
Name,L-arabinopyranose
Memory address,0x01139847d0
Formula,C5H10O5
Compartment,c
In 3 reaction(s),"ARABR, RXN__45__14809, RXN__45__14808"


In [329]:
model.add_boundary(model.metabolites.arab__L_c,type='exchange')
model.reactions.EX_arab__L_c.bounds= -1000, 1000
model.optimize().f

20.571122086897386

In [330]:
model.reactions.EX_arab__L_c.bounds= 0, 1000

In [331]:
model.metabolites.abt__D_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__CPD__45__355__47____47__CPD__45__355__46__17__46__ at 0x113e83810>})

In [332]:
model.metabolites.abt__D_c.reactions

frozenset({<Reaction _1__46__1__46__1__46__250__45__RXN at 0x113526d10>,
           <Reaction TRANS__45__RXNTFS__45__2__45__CPD__45__355__47____47__CPD__45__355__46__17__46__ at 0x113e83810>})

In [333]:
model.reactions._1__46__1__46__1__46__250__45__RXN

Reaction identifier,_1__46__1__46__1__46__250__45__RXN
Name,1.1.1.250-RXN
Memory address,0x0113526d10
Stoichiometry,abt__D_c + nad_c <=> h_c + nadh_c + rbl__D_c D-arabitol + NAD+ <=> H+ + NADH + D-ribulose
GPR,UMAG_11256 or UMAG_00108
Lower bound,-1000.0
Upper bound,1000.0


In [334]:
model.add_boundary(model.metabolites.abt__D_e,type='exchange')
model.reactions.EX_abt__D_e.bounds= -1000, 1000
model.optimize().f

17.132985330696947

In [335]:
model.reactions.EX_abt__D_e.bounds= 0, 1000

In [336]:
for rxn in model.metabolites.fru_e.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__RXNTFS__45__2__45__CPD__45__15382__47____47__CPD__45__15382__46__21__46__: fru_e --> fru_c 0.0 1000.0
TRANS__45__RXNTFS__45__32__45__CPD__45__15382__47____47__CPD__45__15382__46__21__46__: fru_e --> fru_c 0.0 1000.0


In [337]:
model.add_boundary(model.metabolites.fru_e,type='exchange')
model.reactions.EX_fru_e.bounds= -1000, 1000
model.optimize().f

25.655562330422047

In [338]:
model.reactions.EX_fru_e.bounds= 0, 1000

In [339]:
for rxn in model.metabolites.gal_bD_e.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__RXNTFS__45__32__45__GALACTOSE__47____47__GALACTOSE__46__21__46__: gal_bD_e --> gal_bD_c 0.0 1000.0
TRANS__45__RXNTFS__45__50__45__GALACTOSE__47____47__GALACTOSE__46__21__46__: gal_bD_e --> gal_bD_c 0.0 1000.0
TRANS__45__RXNTFS__45__2__45__GALACTOSE__47____47__GALACTOSE__46__21__46__: gal_bD_e --> gal_bD_c 0.0 1000.0


In [340]:
model.add_boundary(model.metabolites.gal_bD_e,type='exchange')
model.reactions.EX_gal_bD_e.bounds= -1000, 1000
model.optimize().f

25.655562330413193

In [341]:
model.reactions.EX_gal_bD_e.bounds= 0, 1000

In [344]:
# model.metabolites.galur_c

In [345]:
model.metabolites.glcn_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__58__45__GLUCONATE__47__PROTON__47____47__GLUCONATE__47__PROTON__46__35__46__ at 0x113422150>,
           <Reaction TRANS__45__RXNTFS__45__7__45__GLUCONATE__47____47__GLUCONATE__46__21__46__ at 0x113ebff50>,
           <Reaction TRANS__45__RXNTFS__45__8__45__GLUCONATE__47____47__GLUCONATE__46__21__46__ at 0x113ee7fd0>,
           <Reaction TRANS__45__RXNTFS__45__1__45__GLUCONATE__47____47__GLUCONATE__46__21__46__ at 0x113fdd290>})

In [346]:
model.add_boundary(model.metabolites.glcn_e,type='exchange')
model.reactions.EX_glcn_e.bounds= -1000, 1000 
model.optimize().f

14.596706351633015

In [347]:
model.reactions.EX_glcn_e.bounds= 0, 1000

In [348]:
model.metabolites.gam6p_c.reactions

frozenset({<Reaction ACGAM6PS at 0x113599ad0>,
           <Reaction L__45__GLN__45__FRUCT__45__6__45__P__45__AMINOTRANS__45__RXN__45__FRUCTOSE__45__6P__47__GLN__47____47__CPD__45__13469__47__GLT__46__31__46__ at 0x113a9fc90>,
           <Reaction GLUCOSAMINE__45__6__45__P__45__DEAMIN__45__RXN at 0x114111650>})

In [349]:
model.add_boundary(model.metabolites.gam6p_c,type='demand')

Reaction identifier,DM_gam6p_c
Name,alpha-D-glucosamine 6-phosphate demand
Memory address,0x0119152750
Stoichiometry,gam6p_c --> alpha-D-glucosamine 6-phosphate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [350]:
model.reactions.DM_gam6p_c.bounds= -1000, 1000
model.optimize().f

36.55793692286508

In [351]:
model.reactions.DM_gam6p_c.bounds= 0, 1000

In [352]:
model.metabolites.dha_c.reactions

frozenset({<Reaction GLYCDy at 0x1132915d0>, <Reaction DHAK at 0x113526fd0>})

In [353]:
model.add_boundary(model.metabolites.dha_c,type="demand")
model.reactions.DM_dha_c.bounds= -1000, 1000
model.optimize().f

20.57112208689847

In [354]:
model.reactions.DM_dha_c.bounds= 0, 1000

In [355]:
model.metabolites.mnl_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__MANNITOL__47____47__MANNITOL__46__19__46__ at 0x113e83510>,
           <Reaction EX_mnl_e at 0x1141e3490>})

In [356]:
model.reactions.EX_mnl_e.bounds= -1000, 1000
model.optimize().f

17.998360227637267

In [357]:
model.reactions.EX_mnl_e.bounds= 0, 1000

In [358]:
model.add_boundary(model.metabolites.man_c,type='demand')

Reaction identifier,DM_man_c
Name,D-mannopyranose demand
Memory address,0x0113e46710
Stoichiometry,man_c --> D-mannopyranose -->
GPR,
Lower bound,0
Upper bound,1000.0


In [359]:
model.reactions.DM_man_c.bounds= -1000, 1000
model.optimize().f

6.353283488331506e-14

In [360]:
model.reactions.DM_man_c.bounds= 0, 1000

In [361]:
model.add_boundary(model.metabolites.raffin_c,type='demand')
model.reactions.DM_raffin_c.bounds= -1000, 1000
model.optimize().f

28.530128075859636

In [362]:
model.reactions.DM_raffin_c.bounds= 0, 1000

In [363]:
model.add_boundary(model.metabolites.rib__D_c,type='demand')
model.reactions.DM_rib__D_c.bounds= -1000, 1000
model.optimize().f

9.620238265006948

In [364]:
model.reactions.DM_rib__D_c.bounds= 0, 1000

In [365]:
model.metabolites.tre_c.reactions

frozenset({<Reaction TRE6PP at 0x113be1d90>,
           <Reaction TREHALA__45__RXN at 0x114132810>})

In [366]:
model.add_boundary(model.metabolites.tre_c,type='demand')
model.reactions.DM_tre_c.bounds= -1000, 1000
model.optimize().f

26.885768723043714

In [367]:
model.reactions.DM_tre_c.bounds=0,1000

In [368]:
model.metabolites.xyl__D_c.reactions

frozenset({<Reaction RXN__45__14503 at 0x113570bd0>,
           <Reaction XYLR at 0x113a40550>})

In [369]:
model.add_boundary(model.metabolites.xyl__D_c,type='demand')
model.reactions.DM_xyl__D_c.bounds = -1000, 1000
model.optimize().f

20.5711220868987

In [370]:
model.reactions.DM_xyl__D_c.bounds=0,1000

In [371]:
model.metabolites.fum_m

Metabolite identifier,fum_m
Name,fumarate
Memory address,0x011376f510
Formula,C4H2O4
Compartment,m
In 3 reaction(s),"SUCDH_q6_m, FUMm, TRANS__45__FUMARATE__45__SUCCINATE__45__RXN"


In [372]:
for rxn in model.metabolites.fum_c.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__FUMARATE__45__SUCCINATE__45__RXN: fum_c + succ_m <=> fum_m + succ_c -1000.0 1000.0
FUMAC: 4fumacac_c + h2o_c --> acac_c + fum_c + h_c 0.0 1000.0
ADSL2r: 25aics_c --> aicar_c + fum_c 0.0 1000.0
TRANS__45__RXNTFS__45__1__45__FUM__47____47__FUM__46__9__46__: fum_e <=> fum_c -1000.0 1000.0
ARGSL: argsuc_c --> arg__L_c + fum_c 0.0 1000.0
ADSL1r: dcamp_c --> amp_c + fum_c 0.0 1000.0
TRANS__45__RXNTFS__45__7__45__FUM__47____47__FUM__46__9__46__: fum_e --> fum_c 0.0 1000.0
RXN__45__22: CANAVANINOSUCCINATE_c <=> CANAVANINE_c + fum_c -1000.0 1000.0


In [373]:
model.reactions.EX_fum_e.bounds = -1000, 1000
model.optimize().f

3.719203327098377e-13

In [374]:
model.reactions.EX_fum_e.bounds=0,1000

In [375]:
model.metabolites.get_by_id('4abut_e').reactions

frozenset({<Reaction ABUTt2rL at 0x1140a1050>})

In [376]:
model.reactions.ABUTt2rL

Reaction identifier,ABUTt2rL
Name,TRANS-RXN-57
Memory address,0x01140a1050
Stoichiometry,4abut_e + h_e --> 4abut_c + h_c 4-aminobutanoate + H+ --> 4-aminobutanoate + H+
GPR,UMAG_03522
Lower bound,0.0
Upper bound,1000.0


In [377]:
model.add_boundary(model.metabolites.get_by_id('4abut_e'),type='exchange')
model.reactions.EX_4abut_e.bounds = -1000, 1000
model.optimize().f

0.10831086604935555

In [378]:
model.reactions.EX_4abut_e.bounds=0,1000

In [383]:
model.metabolites.xylt_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__XYLITOL__47____47__XYLITOL__46__17__46__ at 0x113e83910>,
           <Reaction EX_xylt_e at 0x1141e3310>})

In [384]:
model.reactions.EX_xylt_e.bounds = -1000, 1000
model.optimize().f

15.424619215115312

In [385]:
model.reactions.EX_xylt_e.bounds=0,1000

In [386]:
model.add_boundary(model.metabolites.get_by_id('sucr_c'),type='exchange')
model.reactions.EX_sucr_c.bounds = -1000, 1000
model.optimize().f

28.5301280758596

In [387]:
model.reactions.EX_sucr_c.bounds=0,1000

In [388]:
for rxn in model.metabolites.succ_e.reactions:
        print rxn,rxn.lower_bound,rxn.upper_bound

TRANS__45__RXNTFS__45__1__45__SUC__47____47__SUC__46__9__46__: succ_e --> succ_c 0.0 1000.0
TRANS__45__RXNTFS__45__7__45__SUC__47____47__SUC__46__9__46__: succ_e --> succ_c 0.0 1000.0
EX_succ_e: succ_e -->  0.0 1000.0


In [389]:
model.reactions.EX_succ_e.bounds = -1000, 1000
model.optimize().f

4.3228276683908544e-14

In [390]:
model.reactions.EX_succ_e.bounds=0,1000

In [391]:
model.metabolites.stys_c.reactions

frozenset({<Reaction RXN__45__11501 at 0x113ce5950>})

In [392]:
model.add_boundary(model.metabolites.get_by_id('stys_c'),type='demand')
model.reactions.DM_stys_c.bounds = -1000, 1000
model.optimize().f

28.53012807586

In [393]:
model.reactions.DM_stys_c.bounds=0,1000

In [394]:
model.add_boundary(model.metabolites.get_by_id('quin_kt_e'),type='exchange')
model.reactions.EX_quin_kt_e.bounds = -1000, 1000
model.optimize().f

0.3608864091982009

In [395]:
model.reactions.EX_quin_kt_e.bounds=0,1000

In [396]:
model.reactions.EX_pyr_e.bounds = -1000, 1000
model.optimize().f

5.950717070062526

In [397]:
model.reactions.EX_pyr_e.bounds=0,1000

In [398]:
model.metabolites.acgam_c.reactions

frozenset({<Reaction _3__46__2__46__1__46__17__45__RXN at 0x113585750>,
           <Reaction RXN__45__12626 at 0x113e5e6d0>,
           <Reaction AHEXASE3 at 0x113e5e790>})

In [399]:
model.add_boundary(model.metabolites.get_by_id('acgam_c'),type='exchange')
model.reactions.EX_acgam_c.bounds = -1000, 1000
model.optimize().f

-5.2093678907219356e-15

In [400]:
model.reactions.EX_acgam_c.bounds=0,1000

In [401]:
model.metabolites.inost_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__20 at 0x113f5f550>,
           <Reaction EX_inost_e at 0x1141e3110>})

In [402]:
model.reactions.EX_inost_e.bounds = -1000, 1000
model.optimize().f

-5.2093678907219356e-15

In [403]:
model.reactions.EX_inost_e.bounds=0,1000

In [404]:
model.add_boundary(model.metabolites.get_by_id('ppa_e'),type='exchange')
model.reactions.EX_ppa_e.bounds = -1000, 1000
model.optimize().f

0.35146234696009604

In [405]:
model.reactions.EX_ppa_e.bounds=0,1000

In [406]:
model.metabolites.malttr_c.reactions

frozenset({<Reaction MLTG1ly at 0x113570c50>,
           <Reaction AMALT1 at 0x113bbc190>,
           <Reaction RXN0__45__5182 at 0x113efc450>})

In [407]:
model.add_boundary(model.metabolites.get_by_id('malttr_c'),type='demand')

Reaction identifier,DM_malttr_c
Name,maltotriose demand
Memory address,0x011915fb10
Stoichiometry,malttr_c --> maltotriose -->
GPR,
Lower bound,0
Upper bound,1000.0


In [408]:
model.reactions.DM_malttr_c.bounds = -1000, 1000
model.optimize().f

35.43776540610921

In [409]:
model.reactions.DM_malttr_c.bounds=0,1000

In [410]:
model.reactions.EX_val__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_val__L_e.bounds=0,1000

-3.2102170918e-14


In [411]:
model.reactions.EX_thr__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_thr__L_e.bounds=0,1000

0.108905366028


In [412]:
model.add_boundary(model.metabolites.get_by_id('srb__L_c'),type='demand')
model.reactions.DM_srb__L_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_srb__L_c.bounds=0,1000

25.6555623304


In [413]:
model.reactions.EX_ser__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_ser__L_e.bounds=0,1000

3.27974971051


In [414]:
model.reactions.EX_pro__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_pro__L_e.bounds=0,1000

6.1453496393


In [415]:
model.reactions.EX_phe__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_phe__L_e.bounds=0,1000

6.53412397981


In [416]:
model.metabolites.orn_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__33__45__L__45__ORNITHINE__47__PROTON__47____47__L__45__ORNITHINE__47__PROTON__46__39__46__ at 0x11406f290>})

In [417]:
model.add_boundary(model.metabolites.get_by_id('orn_e'),type='exchange')

Reaction identifier,EX_orn_e
Name,L-ornithine exchange
Memory address,0x0113e26490
Stoichiometry,orn_e <=> L-ornithine <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [418]:
model.reactions.EX_orn_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_orn_e.bounds=0,1000

0.446575502743


In [419]:
model.reactions.EX_mal__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_mal__L_e.bounds=0,1000

2.70928480259e-13


In [420]:
model.reactions.EX_lys__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_lys__L_e.bounds=0,1000

0.437070005066


In [421]:
model.reactions.EX_leu__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_leu__L_e.bounds=0,1000

2.15132158117


In [422]:
model.metabolites.lac__L_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__7__45__L__45__LACTATE__47____47__L__45__LACTATE__46__21__46__ at 0x113eaa850>,
           <Reaction TRANS__45__RXNTFS__45__1__45__L__45__LACTATE__47____47__L__45__LACTATE__46__21__46__ at 0x113fc9690>})

In [423]:
model.add_boundary(model.metabolites.get_by_id('lac__L_e'),type='exchange')

Reaction identifier,EX_lac__L_e
Name,(S)-lactate exchange
Memory address,0x01137ef350
Stoichiometry,lac__L_e <=> (S)-lactate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [424]:
model.reactions.EX_lac__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_lac__L_e.bounds=0,1000

2.63864618678e-14


In [425]:
model.reactions.EX_ile__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_ile__L_e.bounds=0,1000

2.39061971245e-14


In [426]:
model.metabolites.hom__L_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__33__45__HOMO__45__SER__47__PROTON__47____47__HOMO__45__SER__47__PROTON__46__33__46__ at 0x11406fd10>})

In [427]:
model.add_boundary(model.metabolites.get_by_id('hom__L_e'),type='exchange')

Reaction identifier,EX_hom__L_e
Name,L-homoserine exchange
Memory address,0x011915f5d0
Stoichiometry,hom__L_e <=> L-homoserine <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [428]:
model.reactions.EX_hom__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_hom__L_e.bounds=0,1000

0.109100224266


In [429]:
model.reactions.EX_his__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_his__L_e.bounds=0,1000

-7.80708416775e-15


In [430]:
model.reactions.EX_gln__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_gln__L_e.bounds=0,1000

0.446575502744


In [431]:
model.add_boundary(model.metabolites.get_by_id('glu__L_e'),type='exchange')

Reaction identifier,EX_glu__L_e
Name,L-glutamate exchange
Memory address,0x01133a5cd0
Stoichiometry,glu__L_e <=> L-glutamate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [432]:
model.reactions.EX_glu__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_glu__L_e.bounds=0,1000

1.94774310137


In [433]:
model.metabolites.CPD__45__330_c.reactions

frozenset({<Reaction RXN__45__11153 at 0x1135999d0>,
           <Reaction RXN__45__11152 at 0x113599a90>})

In [434]:
model.add_boundary(model.metabolites.get_by_id('CPD__45__330_c'),type='exchange')

Reaction identifier,EX_CPD__45__330_c
Name,"L-galactono-1,4-lactone exchange"
Memory address,0x0113ba9ed0
Stoichiometry,"CPD__45__330_c <=> L-galactono-1,4-lactone <=>"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [435]:
model.reactions.EX_CPD__45__330_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_CPD__45__330_c.bounds=0,1000

-7.84679891664e-14


In [436]:
model.metabolites.erythol_c.reactions

frozenset({<Reaction ER at 0x1141e3a50>,
           <Reaction DM_erythol_c at 0x1141e3a90>,
           <Reaction EMT1 at 0x1141e3b10>})

In [437]:
model.reactions.DM_erythol_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_erythol_c.bounds=0,1000

-7.84679891664e-14


In [438]:
model.reactions.EX_asn__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_asn__L_e.bounds=0,1000

-7.84679891664e-14


In [439]:
model.metabolites.abt_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__L__45__ARABITOL__47____47__L__45__ARABITOL__46__23__46__ at 0x113e83a10>})

In [440]:
model.add_boundary(model.metabolites.get_by_id('abt_e'),type='exchange')
model.reactions.EX_abt_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_abt_e.bounds=0,1000

15.4246192151


In [441]:
model.metabolites.arab__L_c.reactions

frozenset({<Reaction ARABR at 0x113570390>,
           <Reaction RXN__45__14809 at 0x1135851d0>,
           <Reaction RXN__45__14808 at 0x113585b90>,
           <Reaction EX_arab__L_c at 0x119152f50>})

In [442]:
model.add_boundary(model.metabolites.get_by_id('arab__L_c'),type='demand')
model.reactions.DM_arab__L_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_arab__L_c.bounds=0,1000

20.5711220869


In [443]:
model.reactions.EX_ala__L_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_ala__L_e.bounds=0,1000

3.2520405032


In [444]:
model.metabolites.gly_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__16__45__GLY__47__PROTON__47____47__GLY__47__PROTON__46__23__46__ at 0x113f3ec90>,
           <Reaction TRANS__45__RXNTFS__45__26__45__GLY__47____47__GLY__46__9__46__ at 0x113f86ed0>,
           <Reaction TRANS__45__RXNTFS__45__33__45__GLY__47__PROTON__47____47__GLY__47__PROTON__46__23__46__ at 0x114039950>,
           <Reaction EX_gly_e at 0x1141e3650>})

In [445]:
model.reactions.EX_gly_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_gly_e.bounds=0,1000

9.79090110283e-15


In [446]:
model.metabolites.glyc_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__GLYCEROL__47____47__GLYCEROL__46__19__46__ at 0x113e83210>,
           <Reaction TRANS__45__RXNTFS__45__39 at 0x1140c5e50>,
           <Reaction EX_glyc_e at 0x1141e32d0>})

In [447]:
model.reactions.EX_glyc_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_glyc_e.bounds=0,1000

12.9568821538


# Carbon Sources U. maydis should not be able to grow on!

In [451]:
print model.objective

Maximize
-1.0*BIOMASS_REACTION_reverse_42726 + 1.0*BIOMASS_REACTION


In [452]:
model.reactions.EX_nac_e.bounds = 0, 10

In [453]:
model.optimize().f

4.9171719234846555e-14

In [454]:
model.reactions.EX_nac_e

Reaction identifier,EX_nac_e
Name,Nicotinic acid exchange
Memory address,0x01141e3090
Stoichiometry,nac_e --> nicotinate -->
GPR,
Lower bound,0
Upper bound,10


In [455]:
model.reactions.EX_glc__D_e.bounds=0,1000

In [456]:
model.optimize().f

4.9171719234846555e-14

In [458]:
for rxn in model.reactions:
    if (rxn.id.startswith('EX_') or rxn.id.startswith('DM_')) and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_o2_e: o2_e <=>  -1000.0 1000.0 -5.0
EX_co2_e: co2_e <=>  -1000.0 1000.0 -3.52626675718e-12
EX_nh4_e: nh4_e <=>  -1000.0 1000.0 -10.0
EX_pi_e: pi_e <=>  -1000.0 1000.0 4.9983158425e-13
EX_so4_e: so4_e <=>  -1000.0 1000.0 1.65783302827e-13
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 15.0
EX_na1_e: na1_e <=>  -1000.0 1000.0 3.71959252812e-13
EX_nac_e: nac_e -->  0 10 -1.45661260831e-13
EX_fum_e: fum_e -->  0 1000 -1.51736245209e-13
EX_arg__L_e: arg__L_e -->  0.0 1000.0 5.86451437775e-14
EX_asn__L_e: asn__L_e -->  0 1000 5.0
EX_his__L_e: his__L_e -->  0 1000 -2.21362845581e-13
EX_ac_e: ac_e <=>  -10 1000 -10.0


In [460]:
model.reactions.EX_ac_e.bounds = 0, 1000

In [461]:
model.metabolites.get_by_id('12ppd__R_c').reactions

frozenset({<Reaction LCARR at 0x113e26790>})

In [462]:
model.add_boundary(model.metabolites.get_by_id('12ppd__R_c'),type='demand')

Reaction identifier,DM_12ppd__R_c
Name,"(R)-propane-1,2-diol demand"
Memory address,0x01137e3d50
Stoichiometry,"12ppd__R_c --> (R)-propane-1,2-diol -->"
GPR,
Lower bound,0
Upper bound,1000.0


In [463]:
model.reactions.DM_12ppd__R_c.bounds = -1000, 1000

In [464]:
model.optimize().f

9.277100583309798

In [465]:
model.reactions.DM_12ppd__R_c.bounds=0,1000

In [466]:
model.optimize().f

-7.157255609071358e-15

In [467]:
model.metabolites.dad_2_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__14__45__DEOXYADENOSINE__47__PROTON__47____47__DEOXYADENOSINE__47__PROTON__46__45__46__ at 0x113f3e450>,
           <Reaction TRANS__45__RXNTFS__45__24__45__NA__43____47__DEOXYADENOSINE__47____47__NA__43____47__DEOXYADENOSINE__46__39__46__ at 0x113f86310>,
           <Reaction TRANS__45__RXNTFS__45__27__45__DEOXYADENOSINE__47____47__DEOXYADENOSINE__46__31__46__ at 0x113fa4b50>,
           <Reaction TRANS__45__RXN0__45__468__45__DEOXYADENOSINE__47____47__DEOXYADENOSINE__46__31__46__ at 0x113fb1890>,
           <Reaction TRANS__45__RXNTFS__45__46__45__CYTOSINE__47__DEOXYADENOSINE__47____47__CYTOSINE__47__DEOXYADENOSINE__46__49__46__ at 0x114178250>})

In [468]:
model.add_boundary(model.metabolites.get_by_id('dad_2_e'),type='exchange')

Reaction identifier,EX_dad_2_e
Name,2'-deoxyadenosine exchange
Memory address,0x01137e3990
Stoichiometry,dad_2_e <=> 2'-deoxyadenosine <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [469]:
model.reactions.EX_dad_2_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_dad_2_e.bounds=0,1000

7.58207573327e-14


In [470]:
model.metabolites.drib_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__2__45__2__45__DEOXYRIBOSE__47____47__2__45__DEOXYRIBOSE__46__29__46__ at 0x113e6f7d0>,
           <Reaction TRANS__45__RXNTFS__45__32__45__2__45__DEOXYRIBOSE__47____47__2__45__DEOXYRIBOSE__46__29__46__ at 0x1140290d0>})

In [471]:
model.add_boundary(model.metabolites.get_by_id('drib_e'),type='exchange')
model.reactions.EX_drib_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_drib_e.bounds=0,1000

26.2994351669


In [472]:
model.metabolites.actn__R_c.reactions

frozenset({<Reaction ACTD at 0x113a77610>})

In [473]:
model.add_boundary(model.metabolites.get_by_id('actn__R_c'),type='demand')
model.reactions.DM_actn__R_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_actn__R_c.bounds=0,1000

5.17202850673e-14


In [474]:
model.metabolites.ad_c.reactions

frozenset({<Reaction RXN__45__14728 at 0x1132b78d0>})

In [475]:
model.add_boundary(model.metabolites.get_by_id('ad_c'),type='demand')
model.reactions.DM_ad_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_ad_c.bounds=0,1000

1.16309835495e-15


In [476]:
model.metabolites.acac_c.reactions

frozenset({<Reaction OCOAT1m at 0x113422950>,
           <Reaction FUMAC at 0x113538310>,
           <Reaction HMGLx at 0x113e026d0>,
           <Reaction AACOAT at 0x114157bd0>})

In [477]:
model.add_boundary(model.metabolites.get_by_id('acac_c'),type='demand')
model.reactions.DM_acac_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_acac_c.bounds=0,1000

3.4663348173


In [480]:
model.reactions.EX_glc__D_e.bounds= -10, 1000

In [481]:
model.optimize().f

0.49398438169568304

In [482]:
model.reactions.EX_glc__D_e.bounds=0,1000

In [490]:
model.reactions.EX_nac_e.bounds=0,0

In [491]:
model.optimize().f

-7.776117491218603e-14

In [492]:
model.metabolites.adn_c.reactions

frozenset({<Reaction NTD7pp at 0x11333d4d0>,
           <Reaction RXN__45__7001__45__ADENOSINE__47__ARSENATE__47____47__ADENINE__47__RIBOSE__45__1__45__ARSENATE__46__46__46__ at 0x113af6b90>,
           <Reaction PUNP1m at 0x113b0d250>,
           <Reaction AHCi at 0x113b42050>,
           <Reaction ADENODEAMIN__45__RXN at 0x113bcfa10>,
           <Reaction ADNUC at 0x113d31e10>,
           <Reaction ADNK1m at 0x113d44510>,
           <Reaction TRANS__45__RXNTFS__45__14__45__ADENOSINE__47__PROTON__47____47__ADENOSINE__47__PROTON__46__35__46__ at 0x113f3e1d0>,
           <Reaction TRANS__45__RXNTFS__45__24__45__NA__43____47__ADENOSINE__47____47__NA__43____47__ADENOSINE__46__29__46__ at 0x113f86190>,
           <Reaction TRANS__45__RXNTFS__45__27__45__ADENOSINE__47____47__ADENOSINE__46__21__46__ at 0x113fa49d0>,
           <Reaction TRANS__45__RXN0__45__468__45__ADENOSINE__47____47__ADENOSINE__46__21__46__ at 0x113fb1790>,
           <Reaction TRANS__45__RXNTFS__45__46__45__CYTOSINE__47_

In [493]:
model.add_boundary(model.metabolites.get_by_id('adn_e'),type='exchange')
model.reactions.EX_adn_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_adn_e.bounds=0,1000

-7.26186171954e-15


In [494]:
model.metabolites.rbt_c.reactions

frozenset({<Reaction RBTDG at 0x113ce5550>,
           <Reaction TRANS__45__RXNTFS__45__2__45__RIBITOL__47____47__RIBITOL__46__17__46__ at 0x113e83710>})

In [495]:
model.add_boundary(model.metabolites.get_by_id('rbt_e'),type='exchange')
model.reactions.EX_rbt_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_rbt_e.bounds=0,1000

21.9887848928


In [496]:
model.metabolites.alltt_c

Metabolite identifier,alltt_c
Name,allantoate
Memory address,0x01132ce310
Formula,C4H7N4O4
Compartment,c
In 6 reaction(s),"ATAH, TRANS__45__RXNTFS__45__1__45__ALLANTOATE__47____47__ALLANTOATE__46__23__46__, TRANS__45__RXNTFS__45__9, TRANS__45__RXNTFS__45__58__45__ALLANTOATE__47__PROTON__47____47__ALLANTOATE__47__PROTON__46__37__46__, TRANS__45__RXNTFS__45__8__45__ALLANTOATE__47____47__ALLANTOATE__46__23__46__, TRANS__45__RXNTFS__45__7__45__ALLANTOATE__47____47__ALLANTOATE__46__23__46__"


In [497]:
model.metabolites.dca_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__7__45__CPD__45__3617__47____47__CPD__45__3617__46__19__46__ at 0x113eaa050>,
           <Reaction TRANS__45__RXNTFS__45__1__45__CPD__45__3617__47____47__CPD__45__3617__46__19__46__ at 0x113fc9290>})

In [498]:
model.add_boundary(model.metabolites.get_by_id('dca_e'),type='exchange')
model.reactions.EX_dca_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_dca_e.bounds=0,1000

9.85708102375


In [499]:
model.metabolites.hex2manerythol_c

Metabolite identifier,hex2manerythol_c
Name,hexoyl-2-mannosylerithritol
Memory address,0x0113329f50
Formula,C16H30O10
Compartment,c
In 2 reaction(s),"MAC2, MAC1"


In [500]:
model.metabolites.cit_c.reactions

frozenset({<Reaction ACITL at 0x113e46c90>,
           <Reaction TRANS_Citrate_Malate_Shuttle at 0x1141aba90>})

In [501]:
model.add_boundary(model.metabolites.get_by_id('cit_c'),type='demand')
model.reactions.DM_cit_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_cit_c.bounds=0,1000

-6.72440037741e-15


In [502]:
model.remove_reactions(['DM_cit_c'])

In [503]:
model.metabolites.glyc3p_c.reactions

frozenset({<Reaction GPDDA2pp at 0x113526690>,
           <Reaction GPDDA1pp at 0x1135269d0>,
           <Reaction RXN__45__15045 at 0x113585ad0>,
           <Reaction G3PAT160 at 0x113b6f2d0>,
           <Reaction RXN__45__1381__45__PALMITYL__45__COA__47__GLYCEROL__45__3P__47____47__1__45__PALMITOYLGLYCEROL__45__3__45__PHOSPHATE__47__CO__45__A__46__63__46__ at 0x113b6f3d0>,
           <Reaction RXN__45__13805 at 0x113b6f490>,
           <Reaction G3PT at 0x113bf3bd0>,
           <Reaction GLYKm at 0x113c18250>,
           <Reaction GLYCPDIESTER__45__RXN at 0x113cc4550>,
           <Reaction PGSA160 at 0x113d09610>,
           <Reaction G3PD5 at 0x113d7b1d0>,
           <Reaction G3PD2 at 0x113d7b210>,
           <Reaction G3PD1 at 0x113d7b310>})

In [504]:
model.add_boundary(model.metabolites.get_by_id('glyc3p_c'),type='demand')
model.reactions.DM_glyc3p_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_glyc3p_c.bounds=0,1000

22.0844528617


In [505]:
model.remove_reactions(['DM_glyc3p_c'])

In [506]:
model.optimize().f

1.004178413901429e-15

In [507]:
model.metabolites.crn_c.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__58__45__CARNITINE__47__PROTON__47____47__CARNITINE__47__PROTON__46__35__46__ at 0x1134222d0>,
           <Reaction BBHOX at 0x113e0d790>,
           <Reaction TRANS__45__RXNTFS__45__7__45__CARNITINE__47____47__CARNITINE__46__21__46__ at 0x113ed2190>,
           <Reaction TRANS__45__RXNTFS__45__8__45__CARNITINE__47____47__CARNITINE__46__21__46__ at 0x113efc210>,
           <Reaction TRANS__45__RXNTFS__45__1__45__CARNITINE__47____47__CARNITINE__46__21__46__ at 0x113fdd390>,
           <Reaction CARNITINE__45__O__45__ACETYLTRANSFERASE__45__RXN_CYTOSOLIC at 0x1141bc050>,
           <Reaction TRANS_CARNITINE__45__Translocase at 0x1141bc290>})

In [508]:
model.add_boundary(model.metabolites.get_by_id('crn_e'),type='exchange')
model.reactions.EX_crn_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_crn_e.bounds=0,1000

0.0


In [509]:
for rxn in model.reactions:
    if (rxn.id.startswith('EX_') or rxn.id.startswith('DM_')) and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

In [510]:
model.metabolites.asp__D_e.reactions

frozenset({<Reaction TRANS__45__RXNTFS__45__33__45__CPD__45__302__47__PROTON__47____47__CPD__45__302__47__PROTON__46__31__46__ at 0x114091a10>})

In [511]:
model.add_boundary(model.metabolites.get_by_id('asp__D_e'),type='exchange')
model.reactions.EX_asp__D_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_asp__D_e.bounds=0,1000

0.0


In [512]:
model.metabolites.cellb_c.reactions

frozenset({<Reaction RXN__45__10773 at 0x113da20d0>})

In [513]:
model.add_boundary(model.metabolites.get_by_id('cellb_c'),type='demand')
model.reactions.DM_cellb_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_cellb_c.bounds=0,1000

26.885768723


In [514]:
model.metabolites.get_by_id('5aptn_c').reactions

frozenset({<Reaction RXN__45__8243 at 0x113585e90>})

In [515]:
model.add_boundary(model.metabolites.get_by_id('5aptn_c'),type='demand')
model.reactions.DM_5aptn_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_5aptn_c.bounds=0,1000

-1.79252464574e-14


In [516]:
model.metabolites.g1p_c.reactions

frozenset({<Reaction RXN__45__10639 at 0x113585950>,
           <Reaction UGLT at 0x113ba94d0>,
           <Reaction G1PPpp at 0x113c41490>,
           <Reaction GALUi at 0x113cb4110>,
           <Reaction PGMT at 0x113e0d810>,
           <Reaction RXN0__45__5182 at 0x113efc450>})

In [517]:
model.add_boundary(model.metabolites.get_by_id('g1p_c'),type='demand')
model.reactions.DM_g1p_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_g1p_c.bounds=0,1000

36.2378365311


In [522]:
model.reactions.EX_glc__D_e.change=-10,1000
model.reactions.EX_nac_e.bounds=0,10
print model.optimize().f
model.reactions.EX_glc__D_e.bounds=0,1000

-4.84765594275e-15


In [523]:
model.metabolites.glcur_c.reactions

frozenset({<Reaction RXN__45__14693 at 0x113570a50>,
           <Reaction UDPGNP at 0x113599d90>,
           <Reaction INOSTO at 0x113e0df90>})

In [524]:
model.add_boundary(model.metabolites.get_by_id('glcur_c'),type='demand')
model.reactions.DM_glcur_c.bounds = -1000, 1000
print model.optimize().f
model.reactions.DM_glcur_c.bounds=0,1000

-6.17529029108e-15


In [525]:
for rxn in model.reactions:
    if len(rxn.metabolites.keys()) == 1 and abs(rxn.flux)>0:
        print rxn,rxn.lower_bound,rxn.upper_bound,rxn.flux

EX_co2_e: co2_e <=>  -1000.0 1000.0 -3.97903932026e-13
EX_pi_e: pi_e <=>  -1000.0 1000.0 -1.26797358781e-13
EX_h2o_e: h2o_e <=>  -1000.0 1000.0 4.54747350886e-13
EX_4abz_e: 4abz_e <=>  -1000.0 1000.0 1.87289310912e-14
EX_fum_e: fum_e -->  0 1000 1.34077814961e-13
EX_itacon_e: itacon_e -->  0.0 1000.0 -3.07607080785e-13
EX_asn__L_e: asn__L_e -->  0 1000 -1.56194907445e-13
EX_cys__L_e: cys__L_e -->  0.0 1000.0 5.06373803869e-16
EX_ile__L_e: ile__L_e -->  0 1000 7.10158383475e-16


In [526]:
model.metabolites.mal__D_c.reactions

frozenset({<Reaction MALDDH at 0x113b82cd0>,
           <Reaction TRANS__45__RXNTFS__45__7__45__CPD__45__660__47____47__CPD__45__660__46__17__46__ at 0x113ebf290>,
           <Reaction TRANS__45__RXNTFS__45__1__45__CPD__45__660__47____47__CPD__45__660__46__17__46__ at 0x113fc9b50>,
           <Reaction TRANS__45__RXNTFS__45__53__45__CPD__45__660__47____47__CPD__45__660__46__17__46__ at 0x11419bcd0>})

In [527]:
model.add_boundary(model.metabolites.get_by_id('mal__D_e'),type='exchange')
model.reactions.EX_mal__D_e.bounds = -1000, 1000
print model.optimize().f
model.reactions.EX_mal__D_e.bounds=0,1000

-1.67977723315e-14


In [530]:
model.reactions.EX_nac_e.change=0,0

In [531]:
model.reactions.EX_glc__D_e.bounds=-10,1000

In [532]:
model.optimize().f

0.4939843816957789

In [536]:
import cobra
from os import getcwd

relative_directory = getcwd()
target_filename_json = relative_directory + '/iCL_Conversion4.json'
target_filename_xml = relative_directory + '/iCL_Conversion4.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

In [537]:
for rxn in model.metabolites.mal__D_c.reactions:
    print rxn.reaction

mal__D_e --> mal__D_c
mal__D_e --> mal__D_c
mal__D_e --> mal__D_c
mal__D_c + nad_c <=> co2_c + nadh_c + pyr_c
